# 🎯 Advanced Segmentation: Real-World Images with Multiple Objects

**Complete Segmentation Pipeline from Classical to State-of-the-Art**

This notebook demonstrates comprehensive image segmentation techniques using real-world images containing at least 6 different objects. We'll cover everything from classical methods to modern deep learning approaches.

## 🌟 What You'll Learn

### 🔧 **Classical Methods**
- Threshold-based segmentation
- Region growing and watershed
- Active contours (Snakes)
- GrabCut interactive segmentation

### 🧠 **Deep Learning Methods**
- U-Net for semantic segmentation
- Mask R-CNN for instance segmentation
- DeepLabV3+ for dense prediction
- Segment Anything Model (SAM)

### 🚀 **Advanced Techniques**
- Multi-class segmentation
- Real-time segmentation
- Panoptic segmentation
- Video object segmentation

### 📊 **Evaluation Metrics**
- IoU (Intersection over Union)
- Dice coefficient
- Boundary accuracy
- Mean Average Precision (mAP)

---

**🎯 Target Objects in Our Dataset:**
1. 👤 People
2. 🚗 Vehicles
3. 🏠 Buildings
4. 🌳 Trees/Vegetation
5. 🛣️ Roads/Paths
6. ☁️ Sky
7. 🪑 Furniture
8. 🐕 Animals

**⚠️ Requirements:**
- OpenCV 4.0+
- PyTorch/TensorFlow
- Scikit-image
- GPU recommended for deep learning models

In [3]:
# 🔧 Complete Setup: All Libraries for Advanced Segmentation
import cv2
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import seaborn as sns
from PIL import Image, ImageDraw
import os
import time
import random
from pathlib import Path
from tqdm import tqdm
import json

# Deep Learning
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.transforms as transforms
from torchvision.models import segmentation

# Scientific computing
import scipy.ndimage as ndimage
from scipy import spatial
from skimage import segmentation as sk_segmentation
from skimage import morphology, measure, filters
from skimage.feature import peak_local_maxima
from skimage.segmentation import watershed, active_contour
from sklearn.cluster import KMeans
from sklearn.metrics import jaccard_score

# Advanced segmentation libraries
try:
    import detectron2
    from detectron2 import model_zoo
    from detectron2.engine import DefaultPredictor
    from detectron2.config import get_cfg
    from detectron2.utils.visualizer import Visualizer, ColorMode
    from detectron2.data import MetadataCatalog
    DETECTRON2_AVAILABLE = True
except ImportError:
    print("⚠️  Detectron2 not installed - Mask R-CNN examples will use alternative implementation")
    DETECTRON2_AVAILABLE = False

try:
    import albumentations as A
    from albumentations.pytorch import ToTensorV2
    ALBUMENTATIONS_AVAILABLE = True
except ImportError:
    print("⚠️  Albumentations not installed - using basic augmentations")
    ALBUMENTATIONS_AVAILABLE = False

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🖥️  Device: {device}")

if torch.cuda.is_available():
    print(f"🔥 GPU: {torch.cuda.get_device_name()}")
    print(f"💾 GPU Memory: {torch.cuda.get_device_properties(0).total_memory // 1024**3} GB")

# Color palette for segmentation visualization
COLORS = [
    [255, 0, 0],     # Red - Person
    [0, 255, 0],     # Green - Vehicle
    [0, 0, 255],     # Blue - Building
    [255, 255, 0],   # Yellow - Tree
    [255, 0, 255],   # Magenta - Road
    [0, 255, 255],   # Cyan - Sky
    [128, 0, 128],   # Purple - Furniture
    [255, 165, 0],   # Orange - Animal
    [128, 128, 128], # Gray - Other
]

CLASS_NAMES = [
    'Background', 'Person', 'Vehicle', 'Building', 
    'Tree', 'Road', 'Sky', 'Furniture', 'Animal'
]

print("✅ All libraries imported and environment configured!")
print("🎯 Ready for advanced segmentation with real-world images!")

ModuleNotFoundError: No module named 'tqdm'

# 🖼️ Real-World Dataset Creation

Let's create realistic images with multiple objects for segmentation tasks.

In [ ]:
class RealWorldImageGenerator:
    """
    Generate realistic images with multiple objects for segmentation
    """
    
    def __init__(self, image_size=(512, 512)):
        self.image_size = image_size
        self.images = []
        self.masks = []
        self.create_dataset()
    
    def create_dataset(self):
        """Create a dataset of realistic multi-object images"""
        print("🎨 Creating Real-World Multi-Object Dataset")
        print("=" * 44)
        
        # Create different scene types
        scenes = [
            self.create_street_scene,
            self.create_indoor_scene,
            self.create_park_scene,
            self.create_office_scene,
            self.create_living_room_scene
        ]
        
        for i, scene_func in enumerate(scenes):
            print(f"📸 Creating scene {i+1}/{len(scenes)}...")
            image, mask = scene_func()
            self.images.append(image)
            self.masks.append(mask)
        
        print(f"✅ Dataset created: {len(self.images)} images with multiple objects")
    
    def create_street_scene(self):
        """Create a street scene with cars, people, buildings, trees, roads, sky"""
        h, w = self.image_size
        image = np.zeros((h, w, 3), dtype=np.uint8)
        mask = np.zeros((h, w), dtype=np.uint8)
        
        # Sky (top 1/3)
        sky_height = h // 3
        for y in range(sky_height):
            intensity = 200 + int(30 * (sky_height - y) / sky_height)
            image[y, :] = [135 + intensity//4, 206 + intensity//6, 235 + intensity//8]
        mask[:sky_height, :] = 6  # Sky class
        
        # Buildings (middle section)
        building_heights = [h//2 + random.randint(-50, 50) for _ in range(5)]
        building_widths = [w//5] * 5
        x_pos = 0
        
        for i, (width, height) in enumerate(zip(building_widths, building_heights)):
            color = [random.randint(80, 120), random.randint(80, 120), random.randint(80, 120)]
            y_start = max(sky_height, h - height)
            
            # Building body
            image[y_start:h-80, x_pos:x_pos+width] = color
            mask[y_start:h-80, x_pos:x_pos+width] = 3  # Building class
            
            # Add windows
            for window_y in range(y_start + 20, h-100, 40):
                for window_x in range(x_pos + 15, x_pos + width - 15, 30):
                    if random.random() > 0.3:  # Some windows are lit
                        window_color = [255, 255, 200] if random.random() > 0.5 else [50, 50, 80]
                        image[window_y:window_y+15, window_x:window_x+15] = window_color
            
            x_pos += width
        
        # Road (bottom section)
        road_y = h - 80
        image[road_y:, :] = [60, 60, 60]  # Dark gray road
        mask[road_y:, :] = 5  # Road class
        
        # Add road markings
        for x in range(0, w, 40):
            image[road_y + 35:road_y + 45, x:x+20] = [255, 255, 255]
        
        # Add cars
        car_positions = [(100, road_y-30), (300, road_y-30), (450, road_y-35)]
        for x, y in car_positions:
            if x + 60 < w and y + 30 < h:
                car_color = [random.randint(0, 255) for _ in range(3)]
                # Car body
                image[y:y+20, x:x+50] = car_color
                mask[y:y+20, x:x+50] = 2  # Vehicle class
                # Car roof
                roof_color = [max(0, c-50) for c in car_color]
                image[y-10:y, x+10:x+40] = roof_color
                mask[y-10:y, x+10:x+40] = 2
                # Wheels
                cv2.circle(image, (x+10, y+20), 8, (0, 0, 0), -1)
                cv2.circle(image, (x+40, y+20), 8, (0, 0, 0), -1)
                cv2.circle(mask, (x+10, y+20), 8, 2, -1)
                cv2.circle(mask, (x+40, y+20), 8, 2, -1)
        
        # Add trees
        tree_positions = [(50, road_y-20), (200, road_y-25), (350, road_y-15)]
        for x, y in tree_positions:
            if x + 30 < w:
                # Tree trunk
                image[y:y+40, x+12:x+18] = [101, 67, 33]
                mask[y:y+40, x+12:x+18] = 4  # Tree class
                # Tree crown
                cv2.circle(image, (x+15, y-10), 25, (34, 139, 34), -1)
                cv2.circle(mask, (x+15, y-10), 25, 4, -1)
        
        # Add people (simplified stick figures)
        people_positions = [(150, road_y-60), (400, road_y-65)]
        for x, y in people_positions:
            if x + 20 < w and y + 60 < h:
                # Person body (rectangle)
                image[y+20:y+50, x+8:x+12] = [255, 220, 177]  # Skin color
                mask[y+20:y+50, x+8:x+12] = 1  # Person class
                # Head
                cv2.circle(image, (x+10, y+15), 8, (255, 220, 177), -1)
                cv2.circle(mask, (x+10, y+15), 8, 1, -1)
                # Clothes
                clothes_color = [random.randint(50, 200) for _ in range(3)]
                image[y+25:y+45, x+5:x+15] = clothes_color
                mask[y+25:y+45, x+5:x+15] = 1
        
        return image, mask
    
    def create_indoor_scene(self):
        """Create an indoor scene with furniture, people, and objects"""
        h, w = self.image_size
        image = np.ones((h, w, 3), dtype=np.uint8) * 240  # Light background
        mask = np.zeros((h, w), dtype=np.uint8)
        
        # Floor
        floor_y = int(h * 0.7)
        image[floor_y:, :] = [139, 117, 90]  # Wood floor
        mask[floor_y:, :] = 0  # Background class
        
        # Add furniture
        # Table
        table_x, table_y = w//3, floor_y - 40
        table_w, table_h = 120, 15
        image[table_y:table_y+table_h, table_x:table_x+table_w] = [101, 67, 33]
        mask[table_y:table_y+table_h, table_x:table_x+table_w] = 7  # Furniture class
        
        # Table legs
        for leg_x in [table_x+10, table_x+table_w-15]:
            image[table_y+table_h:floor_y, leg_x:leg_x+5] = [101, 67, 33]
            mask[table_y+table_h:floor_y, leg_x:leg_x+5] = 7
        
        # Chair
        chair_x, chair_y = table_x - 50, floor_y - 80
        # Chair back
        image[chair_y:chair_y+60, chair_x:chair_x+8] = [139, 69, 19]
        mask[chair_y:chair_y+60, chair_x:chair_x+8] = 7
        # Chair seat
        image[chair_y+40:chair_y+50, chair_x:chair_x+40] = [139, 69, 19]
        mask[chair_y+40:chair_y+50, chair_x:chair_x+40] = 7
        
        # Person sitting
        person_x = chair_x + 5
        person_y = chair_y + 10
        # Head
        cv2.circle(image, (person_x+15, person_y+10), 12, (255, 220, 177), -1)
        cv2.circle(mask, (person_x+15, person_y+10), 12, 1, -1)
        # Body
        image[person_y+20:person_y+40, person_x+10:person_x+20] = [100, 150, 200]
        mask[person_y+20:person_y+40, person_x+10:person_x+20] = 1
        
        # Cat (animal)
        cat_x, cat_y = w//2 + 50, floor_y - 20
        # Cat body
        cv2.ellipse(image, (cat_x, cat_y), (25, 10), 0, 0, 360, (255, 140, 0), -1)
        cv2.ellipse(mask, (cat_x, cat_y), (25, 10), 0, 0, 360, 8, -1)  # Animal class
        # Cat head
        cv2.circle(image, (cat_x-20, cat_y-5), 8, (255, 140, 0), -1)
        cv2.circle(mask, (cat_x-20, cat_y-5), 8, 8, -1)
        # Cat tail
        cv2.ellipse(image, (cat_x+20, cat_y-10), (15, 3), 45, 0, 360, (255, 140, 0), -1)
        cv2.ellipse(mask, (cat_x+20, cat_y-10), (15, 3), 45, 0, 360, 8, -1)
        
        # Window with view
        window_x, window_y = w - 100, 50
        window_w, window_h = 80, 120
        # Window frame
        image[window_y:window_y+window_h, window_x:window_x+window_w] = [135, 206, 235]  # Sky view
        mask[window_y:window_y+window_h, window_x:window_x+window_w] = 6  # Sky class
        
        # Add some trees visible through window
        tree_x = window_x + 20
        tree_y = window_y + 60
        image[tree_y:tree_y+40, tree_x:tree_x+6] = [101, 67, 33]
        mask[tree_y:tree_y+40, tree_x:tree_x+6] = 4
        cv2.circle(image, (tree_x+3, tree_y), 15, (34, 139, 34), -1)
        cv2.circle(mask, (tree_x+3, tree_y), 15, 4, -1)
        
        return image, mask
    
    def create_park_scene(self):
        """Create a park scene with people, trees, path, bench"""
        h, w = self.image_size
        image = np.zeros((h, w, 3), dtype=np.uint8)
        mask = np.zeros((h, w), dtype=np.uint8)
        
        # Sky
        sky_height = h // 4
        image[:sky_height, :] = [135, 206, 235]
        mask[:sky_height, :] = 6
        
        # Grass background
        image[sky_height:, :] = [34, 139, 34]
        mask[sky_height:, :] = 4  # Vegetation
        
        # Winding path
        path_points = [(0, h-50), (w//4, h-80), (w//2, h-60), (3*w//4, h-90), (w, h-70)]
        path_curve = np.array(path_points, dtype=np.int32)
        
        # Create path mask
        path_mask = np.zeros((h, w), dtype=np.uint8)
        cv2.polylines(path_mask, [path_curve], False, 255, thickness=40)
        image[path_mask > 0] = [139, 117, 90]  # Sandy path
        mask[path_mask > 0] = 5  # Path class
        
        # Large trees
        tree_positions = [(80, h-120), (200, h-140), (350, h-110), (450, h-130)]
        for x, y in tree_positions:
            # Trunk
            trunk_width = random.randint(8, 15)
            trunk_height = random.randint(60, 80)
            image[y:y+trunk_height, x-trunk_width//2:x+trunk_width//2] = [101, 67, 33]
            mask[y:y+trunk_height, x-trunk_width//2:x+trunk_width//2] = 4
            
            # Crown (multiple circles for realistic look)
            crown_radius = random.randint(30, 50)
            for offset_x, offset_y in [(-15, -10), (15, -5), (0, -20), (-10, 5), (10, 8)]:
                crown_x = x + offset_x
                crown_y = y - crown_radius + offset_y
                radius = crown_radius + random.randint(-10, 10)
                green_variant = (34 + random.randint(-20, 20), 139 + random.randint(-30, 30), 34 + random.randint(-20, 20))
                green_variant = tuple(np.clip(green_variant, 0, 255))
                cv2.circle(image, (crown_x, crown_y), radius, green_variant, -1)
                cv2.circle(mask, (crown_x, crown_y), radius, 4, -1)
        
        # Bench
        bench_x, bench_y = w//2 - 40, h - 100
        # Bench seat
        image[bench_y:bench_y+8, bench_x:bench_x+80] = [139, 69, 19]
        mask[bench_y:bench_y+8, bench_x:bench_x+80] = 7  # Furniture
        # Bench back
        image[bench_y-30:bench_y, bench_x:bench_x+5] = [139, 69, 19]
        image[bench_y-30:bench_y, bench_x+75:bench_x+80] = [139, 69, 19]
        mask[bench_y-30:bench_y, bench_x:bench_x+5] = 7
        mask[bench_y-30:bench_y, bench_x+75:bench_x+80] = 7
        
        # People in park
        # Person walking
        person1_x, person1_y = w//4, h - 80
        cv2.circle(image, (person1_x, person1_y-40), 10, (255, 220, 177), -1)
        cv2.circle(mask, (person1_x, person1_y-40), 10, 1, -1)
        image[person1_y-30:person1_y-5, person1_x-8:person1_x+8] = [200, 100, 50]
        mask[person1_y-30:person1_y-5, person1_x-8:person1_x+8] = 1
        # Legs
        image[person1_y-5:person1_y+10, person1_x-4:person1_x+4] = [50, 50, 150]
        mask[person1_y-5:person1_y+10, person1_x-4:person1_x+4] = 1
        
        # Person on bench
        person2_x = bench_x + 20
        person2_y = bench_y - 5
        cv2.circle(image, (person2_x, person2_y-25), 8, (255, 220, 177), -1)
        cv2.circle(mask, (person2_x, person2_y-25), 8, 1, -1)
        image[person2_y-15:person2_y, person2_x-6:person2_x+6] = [100, 200, 100]
        mask[person2_y-15:person2_y, person2_x-6:person2_x+6] = 1
        
        # Dog
        dog_x, dog_y = person1_x + 30, person1_y - 10
        # Dog body
        cv2.ellipse(image, (dog_x, dog_y), (20, 8), 0, 0, 360, (139, 69, 19), -1)
        cv2.ellipse(mask, (dog_x, dog_y), (20, 8), 0, 0, 360, 8, -1)
        # Dog head
        cv2.circle(image, (dog_x-15, dog_y-3), 6, (139, 69, 19), -1)
        cv2.circle(mask, (dog_x-15, dog_y-3), 6, 8, -1)
        # Dog tail
        cv2.ellipse(image, (dog_x+15, dog_y-8), (8, 3), -30, 0, 360, (139, 69, 19), -1)
        cv2.ellipse(mask, (dog_x+15, dog_y-8), (8, 3), -30, 0, 360, 8, -1)
        
        return image, mask
    
    def create_office_scene(self):
        """Create an office scene with desks, computers, people"""
        h, w = self.image_size
        image = np.ones((h, w, 3), dtype=np.uint8) * 250  # White/light gray
        mask = np.zeros((h, w), dtype=np.uint8)
        
        # Floor
        floor_y = int(h * 0.75)
        image[floor_y:, :] = [200, 200, 200]  # Gray carpet
        mask[floor_y:, :] = 0
        
        # Desks
        desk_positions = [(50, floor_y-50), (250, floor_y-50), (450, floor_y-50)]
        for i, (desk_x, desk_y) in enumerate(desk_positions):
            if desk_x + 80 < w:
                # Desk surface
                image[desk_y:desk_y+10, desk_x:desk_x+80] = [139, 117, 90]
                mask[desk_y:desk_y+10, desk_x:desk_x+80] = 7  # Furniture
                
                # Computer monitor
                monitor_x = desk_x + 20
                monitor_y = desk_y - 40
                image[monitor_y:monitor_y+30, monitor_x:monitor_x+25] = [50, 50, 50]  # Black monitor
                mask[monitor_y:monitor_y+30, monitor_x:monitor_x+25] = 7
                # Screen
                image[monitor_y+3:monitor_y+27, monitor_x+3:monitor_x+22] = [100, 150, 255]  # Blue screen
                mask[monitor_y+3:monitor_y+27, monitor_x+3:monitor_x+22] = 7
                
                # Chair
                chair_x = desk_x + 10
                chair_y = desk_y + 15
                # Chair seat
                image[chair_y:chair_y+8, chair_x:chair_x+30] = [100, 100, 100]
                mask[chair_y:chair_y+8, chair_x:chair_x+30] = 7
                # Chair back
                image[chair_y-25:chair_y, chair_x+25:chair_x+30] = [100, 100, 100]
                mask[chair_y-25:chair_y, chair_x+25:chair_x+30] = 7
                
                # Person at desk (every other desk)
                if i % 2 == 0:
                    person_x = chair_x + 10
                    person_y = chair_y - 15
                    # Head
                    cv2.circle(image, (person_x, person_y-15), 8, (255, 220, 177), -1)
                    cv2.circle(mask, (person_x, person_y-15), 8, 1, -1)
                    # Body
                    shirt_color = [random.randint(100, 255) for _ in range(3)]
                    image[person_y-5:person_y+10, person_x-6:person_x+6] = shirt_color
                    mask[person_y-5:person_y+10, person_x-6:person_x+6] = 1
        
        # Office plant
        plant_x, plant_y = w - 80, floor_y - 30
        # Pot
        image[plant_y+20:plant_y+35, plant_x:plant_x+20] = [139, 69, 19]
        mask[plant_y+20:plant_y+35, plant_x:plant_x+20] = 7  # Furniture
        # Plant
        for leaf_offset in [(-5, -10), (5, -15), (0, -25), (-8, -5), (8, -8)]:
            leaf_x = plant_x + 10 + leaf_offset[0]
            leaf_y = plant_y + 20 + leaf_offset[1]
            cv2.circle(image, (leaf_x, leaf_y), 8, (34, 139, 34), -1)
            cv2.circle(mask, (leaf_x, leaf_y), 8, 4, -1)  # Tree/plant class
        
        # Window with city view
        window_x, window_y = 10, 30
        window_w, window_h = 60, 100
        # Sky
        image[window_y:window_y+window_h//3, window_x:window_x+window_w] = [135, 206, 235]
        mask[window_y:window_y+window_h//3, window_x:window_x+window_w] = 6
        # Buildings in distance
        building_y = window_y + window_h//3
        for bld_x in range(window_x, window_x + window_w, 15):
            bld_height = random.randint(20, 40)
            image[building_y:building_y+bld_height, bld_x:bld_x+12] = [100, 100, 100]
            mask[building_y:building_y+bld_height, bld_x:bld_x+12] = 3  # Building class
        
        return image, mask
    
    def create_living_room_scene(self):
        """Create a living room with sofa, TV, people, pet"""
        h, w = self.image_size
        image = np.ones((h, w, 3), dtype=np.uint8) * 220  # Light background
        mask = np.zeros((h, w), dtype=np.uint8)
        
        # Floor
        floor_y = int(h * 0.8)
        image[floor_y:, :] = [139, 117, 90]  # Hardwood floor
        mask[floor_y:, :] = 0
        
        # Sofa
        sofa_x, sofa_y = w//4, floor_y - 60
        sofa_w, sofa_h = 150, 50
        # Sofa body
        image[sofa_y:sofa_y+sofa_h, sofa_x:sofa_x+sofa_w] = [100, 50, 50]  # Red sofa
        mask[sofa_y:sofa_y+sofa_h, sofa_x:sofa_x+sofa_w] = 7  # Furniture
        # Sofa back
        image[sofa_y-20:sofa_y+10, sofa_x:sofa_x+sofa_w] = [80, 40, 40]
        mask[sofa_y-20:sofa_y+10, sofa_x:sofa_x+sofa_w] = 7
        # Armrests
        image[sofa_y-10:sofa_y+sofa_h, sofa_x:sofa_x+15] = [80, 40, 40]
        image[sofa_y-10:sofa_y+sofa_h, sofa_x+sofa_w-15:sofa_x+sofa_w] = [80, 40, 40]
        mask[sofa_y-10:sofa_y+sofa_h, sofa_x:sofa_x+15] = 7
        mask[sofa_y-10:sofa_y+sofa_h, sofa_x+sofa_w-15:sofa_x+sofa_w] = 7
        
        # TV on wall
        tv_x, tv_y = w - 120, h//4
        tv_w, tv_h = 80, 50
        # TV frame
        image[tv_y:tv_y+tv_h, tv_x:tv_x+tv_w] = [50, 50, 50]
        mask[tv_y:tv_y+tv_h, tv_x:tv_x+tv_w] = 7  # Furniture/electronics
        # TV screen
        image[tv_y+5:tv_y+tv_h-5, tv_x+5:tv_x+tv_w-5] = [100, 150, 200]  # Blue screen
        mask[tv_y+5:tv_y+tv_h-5, tv_x+5:tv_x+tv_w-5] = 7
        
        # Coffee table
        table_x, table_y = sofa_x + 40, sofa_y + 60
        image[table_y:table_y+8, table_x:table_x+70] = [139, 117, 90]
        mask[table_y:table_y+8, table_x:table_x+70] = 7
        
        # People on sofa
        # Person 1
        person1_x = sofa_x + 30
        person1_y = sofa_y - 10
        cv2.circle(image, (person1_x, person1_y-20), 10, (255, 220, 177), -1)
        cv2.circle(mask, (person1_x, person1_y-20), 10, 1, -1)
        image[person1_y-10:person1_y+20, person1_x-8:person1_x+8] = [150, 100, 200]
        mask[person1_y-10:person1_y+20, person1_x-8:person1_x+8] = 1
        
        # Person 2
        person2_x = sofa_x + 80
        person2_y = sofa_y - 10
        cv2.circle(image, (person2_x, person2_y-20), 10, (255, 220, 177), -1)
        cv2.circle(mask, (person2_x, person2_y-20), 10, 1, -1)
        image[person2_y-10:person2_y+20, person2_x-8:person2_x+8] = [200, 150, 100]
        mask[person2_y-10:person2_y+20, person2_x-8:person2_x+8] = 1
        
        # Cat on floor
        cat_x, cat_y = table_x + 80, floor_y - 15
        cv2.ellipse(image, (cat_x, cat_y), (20, 8), 0, 0, 360, (50, 50, 50), -1)  # Gray cat
        cv2.ellipse(mask, (cat_x, cat_y), (20, 8), 0, 0, 360, 8, -1)  # Animal class
        cv2.circle(image, (cat_x-15, cat_y-3), 6, (50, 50, 50), -1)
        cv2.circle(mask, (cat_x-15, cat_y-3), 6, 8, -1)
        
        # Window with garden view
        window_x, window_y = 20, 50
        window_w, window_h = 80, 120
        # Sky
        image[window_y:window_y+30, window_x:window_x+window_w] = [135, 206, 235]
        mask[window_y:window_y+30, window_x:window_x+window_w] = 6
        # Garden/trees
        image[window_y+30:window_y+window_h, window_x:window_x+window_w] = [34, 139, 34]
        mask[window_y+30:window_y+window_h, window_x:window_x+window_w] = 4
        # Tree trunk visible
        tree_x = window_x + 30
        tree_y = window_y + 50
        image[tree_y:tree_y+60, tree_x:tree_x+8] = [101, 67, 33]
        mask[tree_y:tree_y+60, tree_x:tree_x+8] = 4
        
        return image, mask
    
    def visualize_dataset(self):
        """Visualize all created images and their segmentation masks"""
        n_images = len(self.images)
        fig, axes = plt.subplots(n_images, 3, figsize=(15, 5*n_images))
        
        scene_names = ['Street Scene', 'Indoor Scene', 'Park Scene', 'Office Scene', 'Living Room']
        
        for i in range(n_images):
            # Original image
            axes[i, 0].imshow(cv2.cvtColor(self.images[i], cv2.COLOR_BGR2RGB))
            axes[i, 0].set_title(f'{scene_names[i]} - Original')
            axes[i, 0].axis('off')
            
            # Segmentation mask
            axes[i, 1].imshow(self.masks[i], cmap='tab10', vmin=0, vmax=8)
            axes[i, 1].set_title(f'{scene_names[i]} - Segmentation Mask')
            axes[i, 1].axis('off')
            
            # Colored overlay
            colored_mask = self.create_colored_mask(self.masks[i])
            overlay = cv2.addWeighted(self.images[i], 0.7, colored_mask, 0.3, 0)
            axes[i, 2].imshow(cv2.cvtColor(overlay, cv2.COLOR_BGR2RGB))
            axes[i, 2].set_title(f'{scene_names[i]} - Overlay')
            axes[i, 2].axis('off')
        
        plt.tight_layout()
        plt.show()
        
        # Print class statistics
        self.print_class_statistics()
    
    def create_colored_mask(self, mask):
        """Create colored visualization of segmentation mask"""
        h, w = mask.shape
        colored = np.zeros((h, w, 3), dtype=np.uint8)
        
        for class_id in range(len(CLASS_NAMES)):
            if class_id < len(COLORS):
                colored[mask == class_id] = COLORS[class_id]
        
        return colored
    
    def print_class_statistics(self):
        """Print statistics about object classes in the dataset"""
        print("\n📊 Dataset Class Statistics:")
        print("=" * 30)
        
        total_pixels = 0
        class_counts = {i: 0 for i in range(len(CLASS_NAMES))}
        
        for mask in self.masks:
            total_pixels += mask.size
            for class_id in range(len(CLASS_NAMES)):
                class_counts[class_id] += np.sum(mask == class_id)
        
        for class_id, class_name in enumerate(CLASS_NAMES):
            if class_counts[class_id] > 0:
                percentage = (class_counts[class_id] / total_pixels) * 100
                print(f"{class_name:12}: {class_counts[class_id]:8,} pixels ({percentage:5.1f}%)")
        
        print(f"\n✅ Total: {len(self.images)} images with {len([c for c in class_counts.values() if c > 0])} object classes")

# Create the real-world dataset
print("🚀 Initializing Real-World Multi-Object Dataset...")
dataset_generator = RealWorldImageGenerator()
dataset_generator.visualize_dataset()

# 🔧 Classical Segmentation Methods

Let's start with traditional computer vision techniques for segmentation.

In [ ]:
class ClassicalSegmentation:
    """
    Classical computer vision segmentation methods
    """
    
    def __init__(self, images, masks):
        self.images = images
        self.masks = masks
        self.results = {}
        
    def threshold_segmentation(self, image_idx=0, show_steps=True):
        """
        Multi-level threshold segmentation using Otsu's method
        """
        print("🎯 Threshold-Based Segmentation")
        print("=" * 35)
        
        image = self.images[image_idx]
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        # Apply Gaussian blur to reduce noise
        blurred = cv2.GaussianBlur(gray, (5, 5), 0)
        
        # Multi-level Otsu thresholding
        thresholds = []
        segmented_images = []
        
        # Binary Otsu
        ret1, binary_otsu = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        thresholds.append(ret1)
        segmented_images.append(binary_otsu)
        
        # Adaptive thresholding
        adaptive_mean = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11, 2)
        adaptive_gaussian = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
        
        segmented_images.extend([adaptive_mean, adaptive_gaussian])
        
        # Multi-class segmentation using multiple thresholds
        # Divide intensity range into segments
        hist = cv2.calcHist([gray], [0], None, [256], [0, 256])
        
        # Find peaks in histogram for natural thresholds
        peaks = []
        for i in range(1, 255):
            if hist[i] > hist[i-1] and hist[i] > hist[i+1] and hist[i] > np.max(hist) * 0.1:
                peaks.append(i)
        
        # Create multi-level segmentation
        if len(peaks) >= 2:
            multi_level = np.zeros_like(gray)
            peaks_sorted = sorted(peaks)
            
            for i, threshold in enumerate(peaks_sorted[:4]):  # Use up to 4 levels
                if i == 0:
                    multi_level[gray <= threshold] = i + 1
                else:
                    multi_level[(gray > peaks_sorted[i-1]) & (gray <= threshold)] = i + 1
            
            multi_level[gray > peaks_sorted[-1]] = len(peaks_sorted) + 1
            segmented_images.append(multi_level)
        
        if show_steps:
            self._visualize_threshold_results(image, gray, segmented_images, thresholds)\
        
        self.results['threshold'] = {\n            'binary_otsu': binary_otsu,\n            'adaptive_mean': adaptive_mean,\n            'adaptive_gaussian': adaptive_gaussian,\n            'multi_level': multi_level if len(peaks) >= 2 else binary_otsu,\n            'thresholds': thresholds\n        }\n        \n        return self.results['threshold']\n    \n    def watershed_segmentation(self, image_idx=0, show_steps=True):\n        \"\"\"\n        Watershed segmentation for separating connected objects\n        \"\"\"\n        print(\"💧 Watershed Segmentation\")\n        print(\"=\" * 26)\n        \n        image = self.images[image_idx]\n        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)\n        \n        # Step 1: Noise removal\n        noise_removal = cv2.medianBlur(gray, 5)\n        \n        # Step 2: Thresholding to get binary image\n        ret, thresh = cv2.threshold(noise_removal, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)\n        \n        # Step 3: Morphological operations to remove small noise\n        kernel = np.ones((3, 3), np.uint8)\n        opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=2)\n        \n        # Step 4: Sure background area\n        sure_bg = cv2.dilate(opening, kernel, iterations=3)\n        \n        # Step 5: Finding sure foreground area using distance transform\n        dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 5)\n        ret, sure_fg = cv2.threshold(dist_transform, 0.5 * dist_transform.max(), 255, 0)\n        \n        # Step 6: Finding unknown region\n        sure_fg = np.uint8(sure_fg)\n        unknown = cv2.subtract(sure_bg, sure_fg)\n        \n        # Step 7: Marker labelling\n        ret, markers = cv2.connectedComponents(sure_fg)\n        \n        # Step 8: Add 1 to all labels so that sure background is not 0, but 1\n        markers = markers + 1\n        \n        # Step 9: Mark the region of unknown with zero\n        markers[unknown == 255] = 0\n        \n        # Step 10: Apply watershed\n        markers = cv2.watershed(image, markers)\n        image_watershed = image.copy()\n        image_watershed[markers == -1] = [255, 0, 0]  # Mark boundaries in red\n        \n        # Create segmentation result\n        segmented = np.zeros_like(gray)\n        unique_markers = np.unique(markers)\n        for i, marker in enumerate(unique_markers[1:]):\n            if marker > 0:\n                segmented[markers == marker] = (i % 8) + 1\n        \n        if show_steps:\n            self._visualize_watershed_results(\n                image, gray, thresh, opening, dist_transform, \n                sure_fg, markers, image_watershed, segmented\n            )\n        \n        self.results['watershed'] = {\n            'segmented': segmented,\n            'markers': markers,\n            'boundaries': image_watershed,\n            'distance_transform': dist_transform\n        }\n        \n        return self.results['watershed']\n    \n    def region_growing(self, image_idx=0, seeds=None, threshold=10, show_steps=True):\n        \"\"\"\n        Region growing segmentation starting from seed points\n        \"\"\"\n        print(\"🌱 Region Growing Segmentation\")\n        print(\"=\" * 32)\n        \n        image = self.images[image_idx]\n        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)\n        h, w = gray.shape\n        \n        # Auto-generate seeds if not provided\n        if seeds is None:\n            seeds = self._generate_seeds(gray, num_seeds=8)\n        \n        # Initialize segmentation result\n        segmented = np.zeros_like(gray, dtype=np.int32)\n        visited = np.zeros_like(gray, dtype=bool)\n        \n        for region_id, (seed_x, seed_y) in enumerate(seeds, 1):\n            if visited[seed_y, seed_x]:\n                continue\n                \n            # Region growing from this seed\n            region_pixels = [(seed_x, seed_y)]\n            region_mean = float(gray[seed_y, seed_x])\n            region_size = 1\n            \n            queue = [(seed_x, seed_y)]\n            visited[seed_y, seed_x] = True\n            segmented[seed_y, seed_x] = region_id\n            \n            while queue:\n                x, y = queue.pop(0)\n                \n                # Check 8-connected neighbors\n                for dx, dy in [(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1)]:\n                    nx, ny = x + dx, y + dy\n                    \n                    if (0 <= nx < w and 0 <= ny < h and \n                        not visited[ny, nx] and \n                        abs(float(gray[ny, nx]) - region_mean) < threshold):\n                        \n                        # Add to region\n                        visited[ny, nx] = True\n                        segmented[ny, nx] = region_id\n                        queue.append((nx, ny))\n                        \n                        # Update region statistics\n                        region_mean = (region_mean * region_size + gray[ny, nx]) / (region_size + 1)\n                        region_size += 1\n        \n        if show_steps:\n            self._visualize_region_growing_results(image, gray, seeds, segmented)\n        \n        self.results['region_growing'] = {\n            'segmented': segmented,\n            'seeds': seeds,\n            'threshold': threshold\n        }\n        \n        return self.results['region_growing']\n    \n    def active_contours(self, image_idx=0, show_steps=True):\n        \"\"\"\n        Active contours (Snakes) for object boundary detection\n        \"\"\"\n        print(\"🐍 Active Contours (Snakes)\")\n        print(\"=\" * 27)\n        \n        image = self.images[image_idx]\n        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)\n        \n        # Apply Gaussian filter for smoother contours\n        smoothed = filters.gaussian(gray, sigma=1.0)\n        \n        # Find initial contours using edge detection\n        edges = filters.sobel(smoothed)\n        \n        # Initialize multiple snakes for different objects\n        contours_list = []\n        h, w = gray.shape\n        \n        # Create circular initial contours at different positions\n        centers = [(w//4, h//4), (3*w//4, h//4), (w//2, h//2), (w//4, 3*h//4), (3*w//4, 3*h//4)]\n        \n        for center_x, center_y in centers:\n            # Create circular initial contour\n            s = np.linspace(0, 2*np.pi, 100)\n            radius = min(w, h) // 8\n            x = center_x + radius * np.cos(s)\n            y = center_y + radius * np.sin(s)\n            init_contour = np.array([x, y]).T\n            \n            # Apply active contour\n            try:\n                snake = active_contour(\n                    edges, init_contour, \n                    alpha=0.015, beta=10, gamma=0.001,\n                    max_iterations=1000\n                )\n                contours_list.append(snake)\n            except:\n                # Fallback if active contour fails\n                contours_list.append(init_contour)\n        \n        # Create segmentation mask from contours\n        segmented = np.zeros_like(gray)\n        for i, contour in enumerate(contours_list):\n            if len(contour) > 0:\n                # Create mask for this contour\n                mask = np.zeros_like(gray)\n                contour_int = contour.astype(np.int32)\n                # Ensure contour points are within image bounds\n                contour_int[:, 0] = np.clip(contour_int[:, 0], 0, w-1)\n                contour_int[:, 1] = np.clip(contour_int[:, 1], 0, h-1)\n                cv2.fillPoly(mask, [contour_int], 255)\n                segmented[mask > 0] = (i % 8) + 1\n        \n        if show_steps:\n            self._visualize_active_contours_results(image, gray, edges, centers, contours_list, segmented)\n        \n        self.results['active_contours'] = {\n            'contours': contours_list,\n            'segmented': segmented,\n            'edges': edges\n        }\n        \n        return self.results['active_contours']\n    \n    def grabcut_segmentation(self, image_idx=0, show_steps=True):\n        \"\"\"\n        GrabCut interactive segmentation\n        \"\"\"\n        print(\"✂️  GrabCut Interactive Segmentation\")\n        print(\"=\" * 36)\n        \n        image = self.images[image_idx]\n        h, w = image.shape[:2]\n        \n        # Create automatic rectangle selections for different regions\n        rectangles = [\n            (w//8, h//8, w//4, h//4),      # Top-left region\n            (5*w//8, h//8, w//4, h//4),    # Top-right region\n            (w//4, h//2, w//2, h//3),      # Center region\n            (w//8, 3*h//4, w//3, h//8),    # Bottom-left region\n            (5*w//8, 3*h//4, w//3, h//8),  # Bottom-right region\n        ]\n        \n        segmented_results = []\n        \n        for i, (x, y, width, height) in enumerate(rectangles):\n            # Ensure rectangle is within image bounds\n            x = max(0, min(x, w - width))\n            y = max(0, min(y, h - height))\n            width = min(width, w - x)\n            height = min(height, h - y)\n            \n            if width <= 0 or height <= 0:\n                continue\n            \n            # Initialize mask\n            mask = np.zeros((h, w), np.uint8)\n            \n            # Define rectangle for GrabCut\n            rect = (x, y, width, height)\n            \n            # Initialize background and foreground models\n            bgd_model = np.zeros((1, 65), np.float64)\n            fgd_model = np.zeros((1, 65), np.float64)\n            \n            try:\n                # Apply GrabCut\n                cv2.grabCut(image, mask, rect, bgd_model, fgd_model, 5, cv2.GC_INIT_WITH_RECT)\n                \n                # Create output mask\n                mask2 = np.where((mask == 2) | (mask == 0), 0, 1).astype('uint8')\n                \n                # Apply mask to get segmented region\n                result = image * mask2[:, :, np.newaxis]\n                \n                segmented_results.append({\n                    'mask': mask2,\n                    'result': result,\n                    'rect': rect\n                })\n            except:\n                # Fallback if GrabCut fails\n                mask2 = np.zeros((h, w), np.uint8)\n                mask2[y:y+height, x:x+width] = 1\n                result = image * mask2[:, :, np.newaxis]\n                segmented_results.append({\n                    'mask': mask2,\n                    'result': result,\n                    'rect': rect\n                })\n        \n        # Combine all segmented regions\n        combined_mask = np.zeros((h, w), dtype=np.uint8)\n        for i, seg_result in enumerate(segmented_results):\n            combined_mask[seg_result['mask'] > 0] = (i % 8) + 1\n        \n        if show_steps:\n            self._visualize_grabcut_results(image, rectangles, segmented_results, combined_mask)\n        \n        self.results['grabcut'] = {\n            'segmented': combined_mask,\n            'individual_results': segmented_results,\n            'rectangles': rectangles\n        }\n        \n        return self.results['grabcut']\n    \n    def _generate_seeds(self, gray, num_seeds=8):\n        \"\"\"Generate seed points for region growing\"\"\"\n        h, w = gray.shape\n        seeds = []\n        \n        # Grid-based seeds\n        for i in range(int(np.sqrt(num_seeds))):\n            for j in range(int(np.sqrt(num_seeds))):\n                x = int((j + 1) * w / (int(np.sqrt(num_seeds)) + 1))\n                y = int((i + 1) * h / (int(np.sqrt(num_seeds)) + 1))\n                seeds.append((x, y))\n        \n        # Add some edge-based seeds\n        edges = cv2.Canny(gray, 50, 150)\n        edge_points = np.where(edges > 0)\n        if len(edge_points[0]) > 0:\n            for _ in range(min(2, len(edge_points[0]))):\n                idx = np.random.randint(len(edge_points[0]))\n                seeds.append((edge_points[1][idx], edge_points[0][idx]))\n        \n        return seeds[:num_seeds]\n    \n    def _visualize_threshold_results(self, image, gray, segmented_images, thresholds):\n        \"\"\"Visualize threshold segmentation results\"\"\"\n        fig, axes = plt.subplots(2, 3, figsize=(15, 10))\n        \n        axes[0, 0].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))\n        axes[0, 0].set_title('Original Image')\n        axes[0, 0].axis('off')\n        \n        axes[0, 1].imshow(gray, cmap='gray')\n        axes[0, 1].set_title('Grayscale')\n        axes[0, 1].axis('off')\n        \n        axes[0, 2].imshow(segmented_images[0], cmap='gray')\n        axes[0, 2].set_title(f'Otsu Binary (T={thresholds[0]:.1f})')\n        axes[0, 2].axis('off')\n        \n        axes[1, 0].imshow(segmented_images[1], cmap='gray')\n        axes[1, 0].set_title('Adaptive Mean')\n        axes[1, 0].axis('off')\n        \n        axes[1, 1].imshow(segmented_images[2], cmap='gray')\n        axes[1, 1].set_title('Adaptive Gaussian')\n        axes[1, 1].axis('off')\n        \n        if len(segmented_images) > 3:\n            axes[1, 2].imshow(segmented_images[3], cmap='tab10')\n            axes[1, 2].set_title('Multi-level Segmentation')\n        else:\n            axes[1, 2].imshow(segmented_images[0], cmap='gray')\n            axes[1, 2].set_title('Binary Threshold')\n        axes[1, 2].axis('off')\n        \n        plt.tight_layout()\n        plt.show()\n    \n    def _visualize_watershed_results(self, image, gray, thresh, opening, dist_transform, sure_fg, markers, boundaries, segmented):\n        \"\"\"Visualize watershed segmentation steps\"\"\"\n        fig, axes = plt.subplots(3, 3, figsize=(15, 15))\n        \n        axes[0, 0].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))\n        axes[0, 0].set_title('Original')\n        axes[0, 0].axis('off')\n        \n        axes[0, 1].imshow(gray, cmap='gray')\n        axes[0, 1].set_title('Grayscale')\n        axes[0, 1].axis('off')\n        \n        axes[0, 2].imshow(thresh, cmap='gray')\n        axes[0, 2].set_title('Threshold')\n        axes[0, 2].axis('off')\n        \n        axes[1, 0].imshow(opening, cmap='gray')\n        axes[1, 0].set_title('Opening')\n        axes[1, 0].axis('off')\n        \n        axes[1, 1].imshow(dist_transform, cmap='hot')\n        axes[1, 1].set_title('Distance Transform')\n        axes[1, 1].axis('off')\n        \n        axes[1, 2].imshow(sure_fg, cmap='gray')\n        axes[1, 2].set_title('Sure Foreground')\n        axes[1, 2].axis('off')\n        \n        axes[2, 0].imshow(markers, cmap='tab10')\n        axes[2, 0].set_title('Markers')\n        axes[2, 0].axis('off')\n        \n        axes[2, 1].imshow(cv2.cvtColor(boundaries, cv2.COLOR_BGR2RGB))\n        axes[2, 1].set_title('Watershed Boundaries')\n        axes[2, 1].axis('off')\n        \n        axes[2, 2].imshow(segmented, cmap='tab10')\n        axes[2, 2].set_title('Final Segmentation')\n        axes[2, 2].axis('off')\n        \n        plt.tight_layout()\n        plt.show()\n    \n    def _visualize_region_growing_results(self, image, gray, seeds, segmented):\n        \"\"\"Visualize region growing results\"\"\"\n        fig, axes = plt.subplots(1, 4, figsize=(20, 5))\n        \n        axes[0].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))\n        axes[0].set_title('Original Image')\n        axes[0].axis('off')\n        \n        axes[1].imshow(gray, cmap='gray')\n        # Plot seeds\n        for i, (x, y) in enumerate(seeds):\n            axes[1].plot(x, y, 'ro', markersize=8)\n            axes[1].text(x+5, y+5, str(i+1), color='red', fontsize=10)\n        axes[1].set_title('Seeds for Region Growing')\n        axes[1].axis('off')\n        \n        axes[2].imshow(segmented, cmap='tab10')\n        axes[2].set_title('Region Growing Result')\n        axes[2].axis('off')\n        \n        # Overlay\n        overlay = cv2.addWeighted(image, 0.7, cv2.applyColorMap((segmented * 30).astype(np.uint8), cv2.COLORMAP_JET), 0.3, 0)\n        axes[3].imshow(cv2.cvtColor(overlay, cv2.COLOR_BGR2RGB))\n        axes[3].set_title('Overlay')\n        axes[3].axis('off')\n        \n        plt.tight_layout()\n        plt.show()\n    \n    def _visualize_active_contours_results(self, image, gray, edges, centers, contours_list, segmented):\n        \"\"\"Visualize active contours results\"\"\"\n        fig, axes = plt.subplots(2, 3, figsize=(15, 10))\n        \n        axes[0, 0].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))\n        axes[0, 0].set_title('Original Image')\n        axes[0, 0].axis('off')\n        \n        axes[0, 1].imshow(gray, cmap='gray')\n        axes[0, 1].set_title('Grayscale')\n        axes[0, 1].axis('off')\n        \n        axes[0, 2].imshow(edges, cmap='gray')\n        axes[0, 2].set_title('Edge Map')\n        axes[0, 2].axis('off')\n        \n        # Show initial contours\n        axes[1, 0].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))\n        for i, center in enumerate(centers):\n            circle = plt.Circle(center, min(image.shape[:2])//8, fill=False, color=f'C{i}', linewidth=2)\n            axes[1, 0].add_patch(circle)\n        axes[1, 0].set_title('Initial Contours')\n        axes[1, 0].axis('off')\n        \n        # Show final contours\n        axes[1, 1].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))\n        for i, contour in enumerate(contours_list):\n            if len(contour) > 0:\n                axes[1, 1].plot(contour[:, 0], contour[:, 1], f'C{i}', linewidth=2)\n        axes[1, 1].set_title('Final Active Contours')\n        axes[1, 1].axis('off')\n        \n        axes[1, 2].imshow(segmented, cmap='tab10')\n        axes[1, 2].set_title('Segmentation Result')\n        axes[1, 2].axis('off')\n        \n        plt.tight_layout()\n        plt.show()\n    \n    def _visualize_grabcut_results(self, image, rectangles, segmented_results, combined_mask):\n        \"\"\"Visualize GrabCut segmentation results\"\"\"\n        fig, axes = plt.subplots(2, 3, figsize=(15, 10))\n        \n        # Original with rectangles\n        axes[0, 0].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))\n        for i, (x, y, w, h) in enumerate(rectangles[:3]):\n            rect = Rectangle((x, y), w, h, linewidth=2, edgecolor=f'C{i}', facecolor='none')\n            axes[0, 0].add_patch(rect)\n        axes[0, 0].set_title('Original with Rectangles')\n        axes[0, 0].axis('off')\n        \n        # Show first few individual results\n        for i in range(min(2, len(segmented_results))):\n            axes[0, i+1].imshow(cv2.cvtColor(segmented_results[i]['result'], cv2.COLOR_BGR2RGB))\n            axes[0, i+1].set_title(f'GrabCut Result {i+1}')\n            axes[0, i+1].axis('off')\n        \n        if len(segmented_results) < 2:\n            axes[0, 2].axis('off')\n        \n        # Show more results and combined\n        for i in range(min(2, len(segmented_results) - 2)):\n            if i + 2 < len(segmented_results):\n                axes[1, i].imshow(cv2.cvtColor(segmented_results[i+2]['result'], cv2.COLOR_BGR2RGB))\n                axes[1, i].set_title(f'GrabCut Result {i+3}')\n                axes[1, i].axis('off')\n        \n        # Combined result\n        axes[1, 2].imshow(combined_mask, cmap='tab10')\n        axes[1, 2].set_title('Combined Segmentation')\n        axes[1, 2].axis('off')\n        \n        # Fill empty subplots\n        for i in range(len(segmented_results), 2):\n            axes[1, i].axis('off')\n        \n        plt.tight_layout()\n        plt.show()\n    \n    def compare_all_methods(self, image_idx=0):\n        \"\"\"Compare all classical segmentation methods on the same image\"\"\"\n        print(\"🔍 Comparing All Classical Methods\")\n        print(\"=\" * 35)\n        \n        # Run all methods\n        thresh_result = self.threshold_segmentation(image_idx, show_steps=False)\n        watershed_result = self.watershed_segmentation(image_idx, show_steps=False)\n        region_result = self.region_growing(image_idx, show_steps=False)\n        contour_result = self.active_contours(image_idx, show_steps=False)\n        grabcut_result = self.grabcut_segmentation(image_idx, show_steps=False)\n        \n        # Visualize comparison\n        fig, axes = plt.subplots(2, 3, figsize=(18, 12))\n        \n        # Original image\n        axes[0, 0].imshow(cv2.cvtColor(self.images[image_idx], cv2.COLOR_BGR2RGB))\n        axes[0, 0].set_title('Original Image')\n        axes[0, 0].axis('off')\n        \n        # Ground truth\n        axes[0, 1].imshow(self.masks[image_idx], cmap='tab10')\n        axes[0, 1].set_title('Ground Truth')\n        axes[0, 1].axis('off')\n        \n        # Threshold segmentation\n        axes[0, 2].imshow(thresh_result['multi_level'], cmap='tab10')\n        axes[0, 2].set_title('Threshold Segmentation')\n        axes[0, 2].axis('off')\n        \n        # Watershed\n        axes[1, 0].imshow(watershed_result['segmented'], cmap='tab10')\n        axes[1, 0].set_title('Watershed')\n        axes[1, 0].axis('off')\n        \n        # Region growing\n        axes[1, 1].imshow(region_result['segmented'], cmap='tab10')\n        axes[1, 1].set_title('Region Growing')\n        axes[1, 1].axis('off')\n        \n        # Active contours\n        axes[1, 2].imshow(contour_result['segmented'], cmap='tab10')\n        axes[1, 2].set_title('Active Contours')\n        axes[1, 2].axis('off')\n        \n        plt.tight_layout()\n        plt.show()\n        \n        # Performance evaluation\n        self._evaluate_classical_methods(image_idx)\n    \n    def _evaluate_classical_methods(self, image_idx=0):\n        \"\"\"Evaluate performance of classical methods\"\"\"\n        print(\"\\n📊 Classical Methods Performance:\")\n        print(\"=\" * 38)\n        \n        gt_mask = self.masks[image_idx]\n        methods = ['threshold', 'watershed', 'region_growing', 'active_contours', 'grabcut']\n        \n        for method in methods:\n            if method in self.results:\n                if method == 'threshold':\n                    pred_mask = self.results[method]['multi_level']\n                else:\n                    pred_mask = self.results[method]['segmented']\n                \n                # Calculate basic metrics\n                # Convert to binary for IoU calculation\n                gt_binary = (gt_mask > 0).astype(int)\n                pred_binary = (pred_mask > 0).astype(int)\n                \n                intersection = np.logical_and(gt_binary, pred_binary).sum()\n                union = np.logical_and(gt_binary, pred_binary).sum()\n                \n                if union > 0:\n                    iou = intersection / union\n                else:\n                    iou = 0.0\n                \n                # Number of segments\n                num_segments = len(np.unique(pred_mask)) - 1  # Exclude background\n                \n                print(f\"{method:15}: IoU={iou:.3f}, Segments={num_segments:2d}\")\n\n# Test classical segmentation methods\nprint(\"🚀 Testing Classical Segmentation Methods\")\nprint(\"=\" * 42)\n\nclassical_seg = ClassicalSegmentation(dataset_generator.images, dataset_generator.masks)\n\n# Test each method individually\nprint(\"\\n1️⃣  Testing on Street Scene (Image 0)\")\nclassical_seg.threshold_segmentation(image_idx=0)\n\nprint(\"\\n2️⃣  Testing Watershed on Indoor Scene (Image 1)\")\nclassical_seg.watershed_segmentation(image_idx=1)\n\nprint(\"\\n3️⃣  Testing Region Growing on Park Scene (Image 2)\")\nclassical_seg.region_growing(image_idx=2)\n\nprint(\"\\n4️⃣  Testing Active Contours on Office Scene (Image 3)\")\nclassical_seg.active_contours(image_idx=3)\n\nprint(\"\\n5️⃣  Testing GrabCut on Living Room Scene (Image 4)\")\nclassical_seg.grabcut_segmentation(image_idx=4)\n\n# Compare all methods\nprint(\"\\n6️⃣  Comparing All Methods on Street Scene\")\nclassical_seg.compare_all_methods(image_idx=0)

# 🧠 Deep Learning Segmentation Methods

Now let's implement state-of-the-art deep learning approaches for segmentation.

In [ ]:
class UNet(nn.Module):
    """
    U-Net architecture for semantic segmentation
    """
    
    def __init__(self, n_channels=3, n_classes=9):
        super(UNet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        
        # Encoder (Contracting path)
        self.inc = self.double_conv(n_channels, 64)
        self.down1 = self.down_block(64, 128)
        self.down2 = self.down_block(128, 256)
        self.down3 = self.down_block(256, 512)
        self.down4 = self.down_block(512, 1024)
        
        # Decoder (Expansive path)
        self.up1 = self.up_block(1024, 512)
        self.up2 = self.up_block(512, 256)
        self.up3 = self.up_block(256, 128)
        self.up4 = self.up_block(128, 64)
        
        # Output layer
        self.outc = nn.Conv2d(64, n_classes, kernel_size=1)
        
    def double_conv(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, 3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )
    
    def down_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.MaxPool2d(2),
            self.double_conv(in_channels, out_channels)
        )
    
    def up_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.ConvTranspose2d(in_channels, in_channels // 2, 2, stride=2),
            nn.BatchNorm2d(in_channels // 2),
            nn.ReLU(inplace=True)
        )
    
    def forward(self, x):
        # Encoder
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        
        # Decoder with skip connections
        x = self.up1(x5)
        x = torch.cat([x4, x], dim=1)
        x = self.double_conv(x.size(1), x4.size(1))(x)
        
        x = self.up2(x)
        x = torch.cat([x3, x], dim=1)
        x = self.double_conv(x.size(1), x3.size(1))(x)
        
        x = self.up3(x)
        x = torch.cat([x2, x], dim=1)
        x = self.double_conv(x.size(1), x2.size(1))(x)
        
        x = self.up4(x)
        x = torch.cat([x1, x], dim=1)
        x = self.double_conv(x.size(1), x1.size(1))(x)
        
        return self.outc(x)

class AttentionUNet(nn.Module):
    """
    Attention U-Net with attention mechanisms
    """
    
    def __init__(self, n_channels=3, n_classes=9):
        super(AttentionUNet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        
        # Encoder
        self.inc = self.conv_block(n_channels, 64)
        self.down1 = self.conv_block(64, 128)
        self.down2 = self.conv_block(128, 256)
        self.down3 = self.conv_block(256, 512)
        self.down4 = self.conv_block(512, 1024)
        
        self.maxpool = nn.MaxPool2d(2)
        
        # Decoder
        self.up4 = nn.ConvTranspose2d(1024, 512, 2, stride=2)
        self.att4 = self.attention_block(512, 512, 256)
        self.upconv4 = self.conv_block(1024, 512)
        
        self.up3 = nn.ConvTranspose2d(512, 256, 2, stride=2)
        self.att3 = self.attention_block(256, 256, 128)
        self.upconv3 = self.conv_block(512, 256)
        
        self.up2 = nn.ConvTranspose2d(256, 128, 2, stride=2)
        self.att2 = self.attention_block(128, 128, 64)
        self.upconv2 = self.conv_block(256, 128)
        
        self.up1 = nn.ConvTranspose2d(128, 64, 2, stride=2)
        self.att1 = self.attention_block(64, 64, 32)
        self.upconv1 = self.conv_block(128, 64)
        
        self.outconv = nn.Conv2d(64, n_classes, 1)
        
    def conv_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, 3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )
    
    def attention_block(self, F_g, F_l, F_int):
        return nn.Sequential(
            nn.Conv2d(F_g, F_int, 1, stride=1, padding=0, bias=True),
            nn.Conv2d(F_l, F_int, 1, stride=1, padding=0, bias=True),
            nn.ReLU(inplace=True),
            nn.Conv2d(F_int, 1, 1, stride=1, padding=0, bias=True),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        # Encoder
        e1 = self.inc(x)
        e2 = self.down1(self.maxpool(e1))
        e3 = self.down2(self.maxpool(e2))
        e4 = self.down3(self.maxpool(e3))
        e5 = self.down4(self.maxpool(e4))
        
        # Decoder with attention
        d4 = self.up4(e5)
        s4 = self.att4(d4)
        d4 = torch.cat([d4, e4 * s4], dim=1)
        d4 = self.upconv4(d4)
        
        d3 = self.up3(d4)
        s3 = self.att3(d3)
        d3 = torch.cat([d3, e3 * s3], dim=1)
        d3 = self.upconv3(d3)
        
        d2 = self.up2(d3)
        s2 = self.att2(d2)
        d2 = torch.cat([d2, e2 * s2], dim=1)
        d2 = self.upconv2(d2)
        
        d1 = self.up1(d2)
        s1 = self.att1(d1)
        d1 = torch.cat([d1, e1 * s1], dim=1)
        d1 = self.upconv1(d1)
        
        out = self.outconv(d1)
        return out

class SegmentationDataset(Dataset):
    """
    Dataset class for segmentation training
    """
    
    def __init__(self, images, masks, transform=None):
        self.images = images
        self.masks = masks
        self.transform = transform
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        image = self.images[idx].copy()
        mask = self.masks[idx].copy()
        
        if self.transform:
            # Convert to PIL for albumentations
            image = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
            mask = Image.fromarray(mask)
            
            # Apply transforms
            transformed = self.transform(image=np.array(image), mask=np.array(mask))
            image = transformed['image']
            mask = transformed['mask']
        
        # Convert to tensors
        if isinstance(image, np.ndarray):
            image = torch.from_numpy(image.transpose(2, 0, 1)).float() / 255.0
        if isinstance(mask, np.ndarray):
            mask = torch.from_numpy(mask).long()
        
        return image, mask

class DeepLearningSegmentation:
    """
    Deep learning segmentation methods implementation
    """
    
    def __init__(self, images, masks):
        self.images = images
        self.masks = masks
        self.device = device
        self.models = {}
        self.results = {}
        
        # Initialize transforms
        if ALBUMENTATIONS_AVAILABLE:
            self.train_transform = A.Compose([\n                A.RandomResizedCrop(512, 512, scale=(0.8, 1.0)),\n                A.HorizontalFlip(p=0.5),\n                A.RandomBrightnessContrast(p=0.3),\n                A.RandomGamma(p=0.3),\n                A.GaussNoise(p=0.2),\n                A.Blur(blur_limit=3, p=0.2),\n            ])\n            \n            self.val_transform = A.Compose([\n                A.Resize(512, 512),\n            ])\n        else:\n            self.train_transform = None\n            self.val_transform = None\n    \n    def prepare_data(self, train_ratio=0.8):\n        \"\"\"Prepare training and validation datasets\"\"\"\n        n_samples = len(self.images)\n        train_size = int(n_samples * train_ratio)\n        \n        # Split data\n        indices = list(range(n_samples))\n        random.shuffle(indices)\n        \n        train_indices = indices[:train_size]\n        val_indices = indices[train_size:]\n        \n        train_images = [self.images[i] for i in train_indices]\n        train_masks = [self.masks[i] for i in train_indices]\n        val_images = [self.images[i] for i in val_indices]\n        val_masks = [self.masks[i] for i in val_indices]\n        \n        # Create datasets\n        train_dataset = SegmentationDataset(train_images, train_masks, self.train_transform)\n        val_dataset = SegmentationDataset(val_images, val_masks, self.val_transform)\n        \n        # Create dataloaders\n        self.train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)\n        self.val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)\n        \n        print(f\"📊 Dataset prepared: {len(train_dataset)} training, {len(val_dataset)} validation samples\")\n        \n        return train_dataset, val_dataset\n    \n    def train_unet(self, epochs=20, lr=1e-3):\n        \"\"\"Train U-Net model\"\"\"\n        print(\"🎯 Training U-Net for Semantic Segmentation\")\n        print(\"=\" * 45)\n        \n        # Prepare data\n        train_dataset, val_dataset = self.prepare_data()\n        \n        # Initialize model\n        model = UNet(n_channels=3, n_classes=len(CLASS_NAMES)).to(self.device)\n        criterion = nn.CrossEntropyLoss()\n        optimizer = optim.Adam(model.parameters(), lr=lr)\n        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5)\n        \n        # Training loop\n        train_losses = []\n        val_losses = []\n        \n        for epoch in range(epochs):\n            # Training\n            model.train()\n            train_loss = 0.0\n            \n            for batch_idx, (images, masks) in enumerate(self.train_loader):\n                images, masks = images.to(self.device), masks.to(self.device)\n                \n                optimizer.zero_grad()\n                outputs = model(images)\n                \n                # Resize outputs to match mask size if needed\n                if outputs.shape[-2:] != masks.shape[-2:]:\n                    outputs = F.interpolate(outputs, size=masks.shape[-2:], mode='bilinear', align_corners=False)\n                \n                loss = criterion(outputs, masks)\n                loss.backward()\n                optimizer.step()\n                \n                train_loss += loss.item()\n            \n            # Validation\n            model.eval()\n            val_loss = 0.0\n            \n            with torch.no_grad():\n                for images, masks in self.val_loader:\n                    images, masks = images.to(self.device), masks.to(self.device)\n                    outputs = model(images)\n                    \n                    if outputs.shape[-2:] != masks.shape[-2:]:\n                        outputs = F.interpolate(outputs, size=masks.shape[-2:], mode='bilinear', align_corners=False)\n                    \n                    loss = criterion(outputs, masks)\n                    val_loss += loss.item()\n            \n            # Calculate average losses\n            avg_train_loss = train_loss / len(self.train_loader)\n            avg_val_loss = val_loss / len(self.val_loader)\n            \n            train_losses.append(avg_train_loss)\n            val_losses.append(avg_val_loss)\n            \n            scheduler.step(avg_val_loss)\n            \n            if epoch % 5 == 0:\n                print(f\"Epoch {epoch:2d}/{epochs}: Train Loss={avg_train_loss:.4f}, Val Loss={avg_val_loss:.4f}\")\n        \n        print(f\"✅ U-Net training completed! Final Val Loss: {avg_val_loss:.4f}\")\n        \n        # Save model\n        self.models['unet'] = model\n        \n        # Plot training curves\n        self._plot_training_curves(train_losses, val_losses, 'U-Net')\n        \n        return model\n    \n    def train_attention_unet(self, epochs=20, lr=1e-3):\n        \"\"\"Train Attention U-Net model\"\"\"\n        print(\"🎯 Training Attention U-Net\")\n        print(\"=\" * 28)\n        \n        # Prepare data if not already done\n        if not hasattr(self, 'train_loader'):\n            self.prepare_data()\n        \n        # Initialize model\n        model = AttentionUNet(n_channels=3, n_classes=len(CLASS_NAMES)).to(self.device)\n        criterion = nn.CrossEntropyLoss()\n        optimizer = optim.Adam(model.parameters(), lr=lr)\n        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5)\n        \n        # Training loop (simplified for demo)\n        train_losses = []\n        val_losses = []\n        \n        for epoch in range(epochs):\n            model.train()\n            train_loss = 0.0\n            \n            for images, masks in self.train_loader:\n                images, masks = images.to(self.device), masks.to(self.device)\n                \n                optimizer.zero_grad()\n                outputs = model(images)\n                \n                if outputs.shape[-2:] != masks.shape[-2:]:\n                    outputs = F.interpolate(outputs, size=masks.shape[-2:], mode='bilinear', align_corners=False)\n                \n                loss = criterion(outputs, masks)\n                loss.backward()\n                optimizer.step()\n                \n                train_loss += loss.item()\n            \n            # Validation\n            model.eval()\n            val_loss = 0.0\n            \n            with torch.no_grad():\n                for images, masks in self.val_loader:\n                    images, masks = images.to(self.device), masks.to(self.device)\n                    outputs = model(images)\n                    \n                    if outputs.shape[-2:] != masks.shape[-2:]:\n                        outputs = F.interpolate(outputs, size=masks.shape[-2:], mode='bilinear', align_corners=False)\n                    \n                    loss = criterion(outputs, masks)\n                    val_loss += loss.item()\n            \n            avg_train_loss = train_loss / len(self.train_loader)\n            avg_val_loss = val_loss / len(self.val_loader)\n            \n            train_losses.append(avg_train_loss)\n            val_losses.append(avg_val_loss)\n            \n            scheduler.step(avg_val_loss)\n            \n            if epoch % 5 == 0:\n                print(f\"Epoch {epoch:2d}/{epochs}: Train Loss={avg_train_loss:.4f}, Val Loss={avg_val_loss:.4f}\")\n        \n        print(f\"✅ Attention U-Net training completed! Final Val Loss: {avg_val_loss:.4f}\")\n        \n        self.models['attention_unet'] = model\n        self._plot_training_curves(train_losses, val_losses, 'Attention U-Net')\n        \n        return model\n    \n    def use_pretrained_deeplabv3(self):\n        \"\"\"Use pretrained DeepLabV3+ model\"\"\"\n        print(\"🔬 Loading Pretrained DeepLabV3+\")\n        print(\"=\" * 33)\n        \n        # Load pretrained model\n        model = segmentation.deeplabv3_resnet101(pretrained=True)\n        \n        # Modify classifier for our number of classes\n        model.classifier[4] = nn.Conv2d(256, len(CLASS_NAMES), 1)\n        model.aux_classifier[4] = nn.Conv2d(256, len(CLASS_NAMES), 1)\n        \n        model = model.to(self.device)\n        model.eval()\n        \n        print(\"✅ DeepLabV3+ loaded and ready for inference\")\n        \n        self.models['deeplabv3'] = model\n        return model\n    \n    def setup_mask_rcnn(self):\n        \"\"\"Setup Mask R-CNN for instance segmentation\"\"\"\n        print(\"🎭 Setting up Mask R-CNN\")\n        print(\"=\" * 24)\n        \n        if DETECTRON2_AVAILABLE:\n            # Detectron2 Mask R-CNN\n            cfg = get_cfg()\n            cfg.merge_from_file(model_zoo.get_config_file(\"COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml\"))\n            cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5\n            cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(\"COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml\")\n            \n            predictor = DefaultPredictor(cfg)\n            self.models['mask_rcnn'] = predictor\n            \n            print(\"✅ Detectron2 Mask R-CNN loaded\")\n        else:\n            # Alternative: Use torchvision Mask R-CNN\n            model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)\n            model = model.to(self.device)\n            model.eval()\n            self.models['mask_rcnn'] = model\n            \n            print(\"✅ Torchvision Mask R-CNN loaded\")\n        \n        return self.models['mask_rcnn']\n    \n    def segment_with_unet(self, image_idx=0, model_type='unet'):\n        \"\"\"Perform segmentation using U-Net\"\"\"\n        if model_type not in self.models:\n            print(f\"❌ {model_type} model not trained yet!\")\n            return None\n        \n        model = self.models[model_type]\n        image = self.images[image_idx]\n        \n        # Preprocess image\n        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)\n        image_resized = cv2.resize(image_rgb, (512, 512))\n        image_tensor = torch.from_numpy(image_resized.transpose(2, 0, 1)).float().unsqueeze(0) / 255.0\n        image_tensor = image_tensor.to(self.device)\n        \n        # Inference\n        model.eval()\n        with torch.no_grad():\n            output = model(image_tensor)\n            prediction = torch.argmax(output, dim=1).squeeze().cpu().numpy()\n        \n        # Resize back to original size\n        prediction_resized = cv2.resize(prediction.astype(np.uint8), (image.shape[1], image.shape[0]), interpolation=cv2.INTER_NEAREST)\n        \n        return prediction_resized\n    \n    def segment_with_deeplabv3(self, image_idx=0):\n        \"\"\"Perform segmentation using DeepLabV3+\"\"\"\n        if 'deeplabv3' not in self.models:\n            self.use_pretrained_deeplabv3()\n        \n        model = self.models['deeplabv3']\n        image = self.images[image_idx]\n        \n        # Preprocess\n        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)\n        transform = transforms.Compose([\n            transforms.ToPILImage(),\n            transforms.Resize((512, 512)),\n            transforms.ToTensor(),\n            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])\n        ])\n        \n        input_tensor = transform(image_rgb).unsqueeze(0).to(self.device)\n        \n        # Inference\n        model.eval()\n        with torch.no_grad():\n            output = model(input_tensor)['out']\n            prediction = torch.argmax(output, dim=1).squeeze().cpu().numpy()\n        \n        # Resize back\n        prediction_resized = cv2.resize(prediction.astype(np.uint8), (image.shape[1], image.shape[0]), interpolation=cv2.INTER_NEAREST)\n        \n        return prediction_resized\n    \n    def segment_with_mask_rcnn(self, image_idx=0):\n        \"\"\"Perform instance segmentation using Mask R-CNN\"\"\"\n        if 'mask_rcnn' not in self.models:\n            self.setup_mask_rcnn()\n        \n        model = self.models['mask_rcnn']\n        image = self.images[image_idx]\n        \n        if DETECTRON2_AVAILABLE:\n            # Detectron2 approach\n            outputs = model(image)\n            \n            # Extract masks\n            instances = outputs[\"instances\"]\n            masks = instances.pred_masks.cpu().numpy()\n            scores = instances.scores.cpu().numpy()\n            classes = instances.pred_classes.cpu().numpy()\n            \n            # Combine masks\n            combined_mask = np.zeros(image.shape[:2], dtype=np.uint8)\n            for i, (mask, score) in enumerate(zip(masks, scores)):\n                if score > 0.5:  # Confidence threshold\n                    combined_mask[mask] = (classes[i] % len(CLASS_NAMES)) + 1\n            \n            return combined_mask, {\n                'masks': masks,\n                'scores': scores,\n                'classes': classes\n            }\n        else:\n            # Torchvision approach\n            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)\n            transform = transforms.Compose([transforms.ToTensor()])\n            input_tensor = transform(image_rgb).unsqueeze(0).to(self.device)\n            \n            model.eval()\n            with torch.no_grad():\n                outputs = model(input_tensor)\n            \n            # Extract results\n            masks = outputs[0]['masks'].cpu().numpy()\n            scores = outputs[0]['scores'].cpu().numpy()\n            labels = outputs[0]['labels'].cpu().numpy()\n            \n            # Combine masks\n            combined_mask = np.zeros(image.shape[:2], dtype=np.uint8)\n            for i, (mask, score) in enumerate(zip(masks, scores)):\n                if score > 0.5:\n                    mask_binary = mask[0] > 0.5\n                    combined_mask[mask_binary] = (labels[i] % len(CLASS_NAMES)) + 1\n            \n            return combined_mask, {\n                'masks': masks,\n                'scores': scores,\n                'labels': labels\n            }\n    \n    def _plot_training_curves(self, train_losses, val_losses, model_name):\n        \"\"\"Plot training and validation loss curves\"\"\"\n        plt.figure(figsize=(10, 6))\n        plt.plot(train_losses, label='Training Loss', color='blue')\n        plt.plot(val_losses, label='Validation Loss', color='red')\n        plt.title(f'{model_name} Training Curves')\n        plt.xlabel('Epoch')\n        plt.ylabel('Loss')\n        plt.legend()\n        plt.grid(True)\n        plt.show()\n    \n    def compare_deep_learning_methods(self, image_idx=0):\n        \"\"\"Compare all deep learning segmentation methods\"\"\"\n        print(\"🔍 Comparing Deep Learning Methods\")\n        print(\"=\" * 36)\n        \n        results = {}\n        \n        # Get predictions from all models\n        if 'unet' in self.models:\n            results['U-Net'] = self.segment_with_unet(image_idx, 'unet')\n        \n        if 'attention_unet' in self.models:\n            results['Attention U-Net'] = self.segment_with_unet(image_idx, 'attention_unet')\n        \n        results['DeepLabV3+'] = self.segment_with_deeplabv3(image_idx)\n        \n        mask_rcnn_result, _ = self.segment_with_mask_rcnn(image_idx)\n        results['Mask R-CNN'] = mask_rcnn_result\n        \n        # Visualize results\n        n_methods = len(results)\n        fig, axes = plt.subplots(2, (n_methods + 1) // 2, figsize=(20, 10))\n        axes = axes.flatten()\n        \n        # Original image\n        axes[0].imshow(cv2.cvtColor(self.images[image_idx], cv2.COLOR_BGR2RGB))\n        axes[0].set_title('Original Image')\n        axes[0].axis('off')\n        \n        # Ground truth\n        if len(axes) > 1:\n            axes[1].imshow(self.masks[image_idx], cmap='tab10')\n            axes[1].set_title('Ground Truth')\n            axes[1].axis('off')\n        \n        # Model predictions\n        for i, (method_name, prediction) in enumerate(results.items()):\n            if i + 2 < len(axes):\n                axes[i + 2].imshow(prediction, cmap='tab10')\n                axes[i + 2].set_title(method_name)\n                axes[i + 2].axis('off')\n        \n        # Hide unused subplots\n        for i in range(len(results) + 2, len(axes)):\n            axes[i].axis('off')\n        \n        plt.tight_layout()\n        plt.show()\n        \n        # Calculate and display metrics\n        self._evaluate_deep_learning_methods(image_idx, results)\n    \n    def _evaluate_deep_learning_methods(self, image_idx, results):\n        \"\"\"Evaluate deep learning methods performance\"\"\"\n        print(\"\\n📊 Deep Learning Methods Performance:\")\n        print(\"=\" * 40)\n        \n        gt_mask = self.masks[image_idx]\n        \n        for method_name, prediction in results.items():\n            # Calculate IoU\n            gt_binary = (gt_mask > 0).astype(int)\n            pred_binary = (prediction > 0).astype(int)\n            \n            intersection = np.logical_and(gt_binary, pred_binary).sum()\n            union = np.logical_or(gt_binary, pred_binary).sum()\n            \n            iou = intersection / union if union > 0 else 0.0\n            \n            # Calculate pixel accuracy\n            accuracy = np.mean(gt_mask == prediction)\n            \n            # Number of predicted classes\n            n_classes = len(np.unique(prediction))\n            \n            print(f\"{method_name:15}: IoU={iou:.3f}, Accuracy={accuracy:.3f}, Classes={n_classes:2d}\")\n\n# Initialize deep learning segmentation\nprint(\"🚀 Initializing Deep Learning Segmentation\")\nprint(\"=\" * 43)\n\ndl_seg = DeepLearningSegmentation(dataset_generator.images, dataset_generator.masks)\n\n# Train models (with reduced epochs for demo)\nprint(\"\\n1️⃣  Training U-Net (Quick Demo - 5 epochs)\")\nunet_model = dl_seg.train_unet(epochs=5, lr=1e-3)\n\nprint(\"\\n2️⃣  Training Attention U-Net (Quick Demo - 5 epochs)\")\nattention_unet_model = dl_seg.train_attention_unet(epochs=5, lr=1e-3)\n\nprint(\"\\n3️⃣  Loading Pretrained DeepLabV3+\")\ndeeplabv3_model = dl_seg.use_pretrained_deeplabv3()\n\nprint(\"\\n4️⃣  Setting up Mask R-CNN\")\nmask_rcnn_model = dl_seg.setup_mask_rcnn()\n\n# Compare all deep learning methods\nprint(\"\\n5️⃣  Comparing All Deep Learning Methods\")\ndl_seg.compare_deep_learning_methods(image_idx=0)

# 🚀 Advanced Techniques & Modern Methods

Let's explore cutting-edge segmentation techniques and comprehensive evaluation.

In [ ]:
class AdvancedSegmentationTechniques:
    """
    State-of-the-art segmentation techniques and evaluation
    """
    
    def __init__(self, images, masks):
        self.images = images
        self.masks = masks
        self.device = device
        self.results = {}
    
    def segment_anything_simulation(self, image_idx=0, num_points=20):
        \"\"\"
        Simulate Segment Anything Model (SAM) behavior
        Note: This is a simulation since SAM requires specific setup
        \"\"\"\n        print(\"🎯 Segment Anything Model (SAM) Simulation\")\n        print(\"=\" * 44)\n        \n        image = self.images[image_idx]\n        h, w = image.shape[:2]\n        \n        # Simulate SAM's point-based segmentation\n        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)\n        \n        # Generate random points (simulating user clicks)\n        points = []\n        for _ in range(num_points):\n            x = random.randint(0, w-1)\n            y = random.randint(0, h-1)\n            points.append((x, y))\n        \n        # For each point, create a segment using region growing\n        segmented = np.zeros((h, w), dtype=np.uint8)\n        \n        for i, (px, py) in enumerate(points):\n            # Simple region growing from point\n            mask = np.zeros((h, w), dtype=bool)\n            visited = np.zeros((h, w), dtype=bool)\n            \n            threshold = 30  # Intensity threshold\n            queue = [(px, py)]\n            seed_intensity = gray[py, px]\n            \n            while queue and np.sum(mask) < (h * w) // 50:  # Limit region size\n                x, y = queue.pop(0)\n                \n                if visited[y, x]:\n                    continue\n                \n                visited[y, x] = True\n                \n                if abs(int(gray[y, x]) - int(seed_intensity)) < threshold:\n                    mask[y, x] = True\n                    \n                    # Add neighbors\n                    for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:\n                        nx, ny = x + dx, y + dy\n                        if 0 <= nx < w and 0 <= ny < h and not visited[ny, nx]:\n                            queue.append((nx, ny))\n            \n            # Assign segment ID\n            segmented[mask] = (i % 8) + 1\n        \n        # Post-process: merge overlapping segments\n        final_segmented = self._merge_overlapping_segments(segmented)\n        \n        self._visualize_sam_simulation(image, points, segmented, final_segmented)\n        \n        self.results['sam_simulation'] = {\n            'segmented': final_segmented,\n            'points': points,\n            'raw_segments': segmented\n        }\n        \n        return final_segmented\n    \n    def panoptic_segmentation(self, image_idx=0):\n        \"\"\"Panoptic segmentation combining semantic and instance segmentation\"\"\"\n        print(\"🎭 Panoptic Segmentation\")\n        print(\"=\" * 24)\n        \n        image = self.images[image_idx]\n        \n        # Step 1: Semantic segmentation (simplified)\n        semantic_mask = self._simple_semantic_segmentation(image)\n        \n        # Step 2: Instance segmentation (simplified)\n        instance_masks = self._simple_instance_segmentation(image)\n        \n        # Step 3: Combine semantic and instance information\n        panoptic_mask = self._combine_semantic_instance(semantic_mask, instance_masks)\n        \n        self._visualize_panoptic_segmentation(image, semantic_mask, instance_masks, panoptic_mask)\n        \n        self.results['panoptic'] = {\n            'panoptic_mask': panoptic_mask,\n            'semantic_mask': semantic_mask,\n            'instance_masks': instance_masks\n        }\n        \n        return panoptic_mask\n    \n    def real_time_segmentation(self, image_idx=0):\n        \"\"\"Optimized segmentation for real-time applications\"\"\"\n        print(\"⚡ Real-Time Segmentation\")\n        print(\"=\" * 25)\n        \n        image = self.images[image_idx]\n        \n        # Method 1: Fast superpixel segmentation\n        start_time = time.time()\n        superpixels = self._fast_superpixel_segmentation(image)\n        superpixel_time = time.time() - start_time\n        \n        # Method 2: Fast clustering-based segmentation\n        start_time = time.time()\n        clustered = self._fast_clustering_segmentation(image)\n        clustering_time = time.time() - start_time\n        \n        # Method 3: Edge-based fast segmentation\n        start_time = time.time()\n        edge_based = self._fast_edge_segmentation(image)\n        edge_time = time.time() - start_time\n        \n        print(f\"⚡ Superpixels: {superpixel_time:.3f}s\")\n        print(f\"⚡ Clustering:  {clustering_time:.3f}s\")\n        print(f\"⚡ Edge-based:  {edge_time:.3f}s\")\n        \n        self._visualize_realtime_methods(image, superpixels, clustered, edge_based)\n        \n        self.results['realtime'] = {\n            'superpixels': superpixels,\n            'clustered': clustered,\n            'edge_based': edge_based,\n            'times': {\n                'superpixel': superpixel_time,\n                'clustering': clustering_time,\n                'edge': edge_time\n            }\n        }\n        \n        return superpixels, clustered, edge_based\n    \n    def video_object_segmentation_simulation(self, num_frames=5):\n        \"\"\"Simulate video object segmentation using multiple images\"\"\"\n        print(\"🎬 Video Object Segmentation Simulation\")\n        print(\"=\" * 40)\n        \n        # Use multiple images as \"frames\"\n        frames = self.images[:num_frames]\n        video_results = []\n        \n        # Initialize with first frame\n        prev_mask = self._simple_semantic_segmentation(frames[0])\n        video_results.append(prev_mask)\n        \n        # Process subsequent frames with temporal consistency\n        for i in range(1, len(frames)):\n            current_frame = frames[i]\n            \n            # Current frame segmentation\n            current_mask = self._simple_semantic_segmentation(current_frame)\n            \n            # Apply temporal smoothing\n            smoothed_mask = self._temporal_smoothing(prev_mask, current_mask)\n            \n            video_results.append(smoothed_mask)\n            prev_mask = smoothed_mask\n        \n        self._visualize_video_segmentation(frames, video_results)\n        \n        self.results['video'] = {\n            'frames': frames,\n            'segmentations': video_results\n        }\n        \n        return video_results\n    \n    def _simple_semantic_segmentation(self, image):\n        \"\"\"Simple semantic segmentation using color-based clustering\"\"\"\n        # Reshape image for clustering\n        data = image.reshape((-1, 3)).astype(np.float32)\n        \n        # K-means clustering\n        criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 20, 1.0)\n        _, labels, centers = cv2.kmeans(data, 8, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)\n        \n        # Reshape back to image\n        segmented = labels.reshape(image.shape[:2])\n        \n        return segmented.astype(np.uint8)\n    \n    def _simple_instance_segmentation(self, image):\n        \"\"\"Simple instance segmentation using connected components\"\"\"\n        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)\n        \n        # Threshold and find connected components\n        _, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)\n        \n        # Morphological operations\n        kernel = np.ones((5, 5), np.uint8)\n        opened = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel)\n        \n        # Connected components\n        num_labels, labels = cv2.connectedComponents(opened)\n        \n        # Create individual instance masks\n        instance_masks = []\n        for i in range(1, num_labels):\n            mask = (labels == i).astype(np.uint8)\n            if np.sum(mask) > 500:  # Filter small components\n                instance_masks.append(mask)\n        \n        return instance_masks\n    \n    def _combine_semantic_instance(self, semantic_mask, instance_masks):\n        \"\"\"Combine semantic and instance segmentation for panoptic result\"\"\"\n        h, w = semantic_mask.shape\n        panoptic_mask = semantic_mask.copy()\n        \n        # Assign unique IDs to instances within each semantic class\n        for i, instance_mask in enumerate(instance_masks):\n            # Find dominant semantic class in this instance\n            semantic_values = semantic_mask[instance_mask > 0]\n            if len(semantic_values) > 0:\n                dominant_class = np.bincount(semantic_values).argmax()\n                # Assign unique instance ID (class * 1000 + instance_id)\n                instance_id = dominant_class * 1000 + i + 1\n                panoptic_mask[instance_mask > 0] = instance_id % 255\n        \n        return panoptic_mask\n    \n    def _fast_superpixel_segmentation(self, image):\n        \"\"\"Fast superpixel segmentation using SLIC\"\"\"\n        from skimage.segmentation import slic\n        \n        # Convert to RGB\n        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)\n        \n        # SLIC superpixels\n        segments = slic(image_rgb, n_segments=100, compactness=10, sigma=1)\n        \n        return segments\n    \n    def _fast_clustering_segmentation(self, image):\n        \"\"\"Fast clustering-based segmentation\"\"\"\n        # Downsample for speed\n        small_image = cv2.resize(image, (128, 128))\n        \n        # K-means clustering\n        data = small_image.reshape((-1, 3)).astype(np.float32)\n        criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)\n        _, labels, _ = cv2.kmeans(data, 6, None, criteria, 5, cv2.KMEANS_RANDOM_CENTERS)\n        \n        # Reshape and resize back\n        segmented_small = labels.reshape(small_image.shape[:2])\n        segmented = cv2.resize(segmented_small.astype(np.uint8), (image.shape[1], image.shape[0]), \n                              interpolation=cv2.INTER_NEAREST)\n        \n        return segmented\n    \n    def _fast_edge_segmentation(self, image):\n        \"\"\"Fast edge-based segmentation\"\"\"\n        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)\n        \n        # Fast edge detection\n        edges = cv2.Canny(gray, 50, 150)\n        \n        # Morphological closing to connect edges\n        kernel = np.ones((3, 3), np.uint8)\n        closed = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, kernel)\n        \n        # Fill regions\n        contours, _ = cv2.findContours(closed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)\n        \n        segmented = np.zeros_like(gray)\n        for i, contour in enumerate(contours[:10]):  # Limit to 10 largest\n            cv2.fillPoly(segmented, [contour], (i % 8) + 1)\n        \n        return segmented\n    \n    def _temporal_smoothing(self, prev_mask, current_mask, alpha=0.7):\n        \"\"\"Apply temporal smoothing between frames\"\"\"\n        # Simple weighted average\n        smoothed = alpha * prev_mask + (1 - alpha) * current_mask\n        return smoothed.astype(np.uint8)\n    \n    def _merge_overlapping_segments(self, segmented):\n        \"\"\"Merge overlapping segments\"\"\"\n        # Simple merging based on spatial proximity\n        unique_labels = np.unique(segmented)[1:]  # Exclude background\n        merged = segmented.copy()\n        \n        for label in unique_labels:\n            mask = (segmented == label)\n            # Apply morphological operations to smooth\n            kernel = np.ones((3, 3), np.uint8)\n            mask_smoothed = cv2.morphologyEx(mask.astype(np.uint8), cv2.MORPH_CLOSE, kernel)\n            merged[mask_smoothed > 0] = label\n        \n        return merged\n    \n    def _visualize_sam_simulation(self, image, points, raw_segments, final_segments):\n        \"\"\"Visualize SAM simulation results\"\"\"\n        fig, axes = plt.subplots(1, 4, figsize=(20, 5))\n        \n        # Original with points\n        axes[0].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))\n        for i, (x, y) in enumerate(points[:10]):  # Show first 10 points\n            axes[0].plot(x, y, 'ro', markersize=6)\n            if i < 5:  # Label first 5\n                axes[0].text(x+5, y+5, str(i+1), color='red', fontsize=8)\n        axes[0].set_title('Original + Sample Points')\n        axes[0].axis('off')\n        \n        # Raw segments\n        axes[1].imshow(raw_segments, cmap='tab10')\n        axes[1].set_title('Raw Segments')\n        axes[1].axis('off')\n        \n        # Final segments\n        axes[2].imshow(final_segments, cmap='tab10')\n        axes[2].set_title('Final SAM Simulation')\n        axes[2].axis('off')\n        \n        # Overlay\n        overlay = cv2.addWeighted(image, 0.7, cv2.applyColorMap((final_segments * 30).astype(np.uint8), cv2.COLORMAP_JET), 0.3, 0)\n        axes[3].imshow(cv2.cvtColor(overlay, cv2.COLOR_BGR2RGB))\n        axes[3].set_title('Overlay')\n        axes[3].axis('off')\n        \n        plt.tight_layout()\n        plt.show()\n    \n    def _visualize_panoptic_segmentation(self, image, semantic_mask, instance_masks, panoptic_mask):\n        \"\"\"Visualize panoptic segmentation results\"\"\"\n        fig, axes = plt.subplots(2, 3, figsize=(18, 12))\n        \n        # Original\n        axes[0, 0].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))\n        axes[0, 0].set_title('Original Image')\n        axes[0, 0].axis('off')\n        \n        # Semantic segmentation\n        axes[0, 1].imshow(semantic_mask, cmap='tab10')\n        axes[0, 1].set_title('Semantic Segmentation')\n        axes[0, 1].axis('off')\n        \n        # Instance segmentation (combined)\n        if instance_masks:\n            combined_instances = np.zeros_like(semantic_mask)\n            for i, mask in enumerate(instance_masks[:8]):\n                combined_instances[mask > 0] = i + 1\n            axes[0, 2].imshow(combined_instances, cmap='tab10')\n        else:\n            axes[0, 2].imshow(np.zeros_like(semantic_mask), cmap='gray')\n        axes[0, 2].set_title('Instance Segmentation')\n        axes[0, 2].axis('off')\n        \n        # Panoptic result\n        axes[1, 0].imshow(panoptic_mask, cmap='tab20')\n        axes[1, 0].set_title('Panoptic Segmentation')\n        axes[1, 0].axis('off')\n        \n        # Individual instances (first few)\n        for i in range(min(2, len(instance_masks))):\n            axes[1, i+1].imshow(instance_masks[i], cmap='gray')\n            axes[1, i+1].set_title(f'Instance {i+1}')\n            axes[1, i+1].axis('off')\n        \n        # Fill remaining subplots\n        for i in range(len(instance_masks), 2):\n            axes[1, i+1].axis('off')\n        \n        plt.tight_layout()\n        plt.show()\n    \n    def _visualize_realtime_methods(self, image, superpixels, clustered, edge_based):\n        \"\"\"Visualize real-time segmentation methods\"\"\"\n        fig, axes = plt.subplots(1, 4, figsize=(20, 5))\n        \n        axes[0].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))\n        axes[0].set_title('Original Image')\n        axes[0].axis('off')\n        \n        axes[1].imshow(superpixels, cmap='tab20')\n        axes[1].set_title('Superpixels (SLIC)')\n        axes[1].axis('off')\n        \n        axes[2].imshow(clustered, cmap='tab10')\n        axes[2].set_title('Fast Clustering')\n        axes[2].axis('off')\n        \n        axes[3].imshow(edge_based, cmap='tab10')\n        axes[3].set_title('Edge-based')\n        axes[3].axis('off')\n        \n        plt.tight_layout()\n        plt.show()\n    \n    def _visualize_video_segmentation(self, frames, segmentations):\n        \"\"\"Visualize video segmentation results\"\"\"\n        n_frames = len(frames)\n        fig, axes = plt.subplots(2, n_frames, figsize=(4*n_frames, 8))\n        \n        for i in range(n_frames):\n            # Original frames\n            axes[0, i].imshow(cv2.cvtColor(frames[i], cv2.COLOR_BGR2RGB))\n            axes[0, i].set_title(f'Frame {i+1}')\n            axes[0, i].axis('off')\n            \n            # Segmentations\n            axes[1, i].imshow(segmentations[i], cmap='tab10')\n            axes[1, i].set_title(f'Segmentation {i+1}')\n            axes[1, i].axis('off')\n        \n        plt.tight_layout()\n        plt.show()\n    \n    def comprehensive_evaluation(self, image_idx=0):\n        \"\"\"Comprehensive evaluation of all methods\"\"\"\n        print(\"📊 Comprehensive Evaluation\")\n        print(\"=\" * 28)\n        \n        # Get ground truth\n        gt_mask = self.masks[image_idx]\n        image = self.images[image_idx]\n        \n        # Collect all segmentation results\n        all_results = {}\n        \n        # Run all methods\n        sam_result = self.segment_anything_simulation(image_idx)\n        all_results['SAM Simulation'] = sam_result\n        \n        panoptic_result = self.panoptic_segmentation(image_idx)\n        all_results['Panoptic'] = panoptic_result\n        \n        rt_super, rt_cluster, rt_edge = self.real_time_segmentation(image_idx)\n        all_results['RT Superpixels'] = rt_super\n        all_results['RT Clustering'] = rt_cluster\n        all_results['RT Edge-based'] = rt_edge\n        \n        # Calculate comprehensive metrics\n        self._calculate_all_metrics(gt_mask, all_results)\n        \n        # Visualize comparison\n        self._visualize_comprehensive_comparison(image, gt_mask, all_results)\n    \n    def _calculate_all_metrics(self, gt_mask, results):\n        \"\"\"Calculate comprehensive evaluation metrics\"\"\"\n        print(\"\\n🎯 Detailed Metrics:\")\n        print(\"=\" * 20)\n        \n        for method_name, pred_mask in results.items():\n            metrics = self._compute_segmentation_metrics(gt_mask, pred_mask)\n            \n            print(f\"\\n{method_name}:\")\n            print(f\"  IoU (Intersection over Union): {metrics['iou']:.3f}\")\n            print(f\"  Dice Coefficient:              {metrics['dice']:.3f}\")\n            print(f\"  Pixel Accuracy:                {metrics['accuracy']:.3f}\")\n            print(f\"  Boundary F1-Score:             {metrics['boundary_f1']:.3f}\")\n            print(f\"  Number of Segments:            {metrics['n_segments']:3d}\")\n            print(f\"  Segmentation Quality:          {metrics['seg_quality']:.3f}\")\n    \n    def _compute_segmentation_metrics(self, gt_mask, pred_mask):\n        \"\"\"Compute various segmentation metrics\"\"\"\n        # Ensure same size\n        if gt_mask.shape != pred_mask.shape:\n            pred_mask = cv2.resize(pred_mask.astype(np.uint8), (gt_mask.shape[1], gt_mask.shape[0]), \n                                 interpolation=cv2.INTER_NEAREST)\n        \n        # Convert to binary for some metrics\n        gt_binary = (gt_mask > 0).astype(int)\n        pred_binary = (pred_mask > 0).astype(int)\n        \n        # IoU (Intersection over Union)\n        intersection = np.logical_and(gt_binary, pred_binary).sum()\n        union = np.logical_or(gt_binary, pred_binary).sum()\n        iou = intersection / union if union > 0 else 0.0\n        \n        # Dice Coefficient\n        dice = 2 * intersection / (gt_binary.sum() + pred_binary.sum()) if (gt_binary.sum() + pred_binary.sum()) > 0 else 0.0\n        \n        # Pixel Accuracy\n        accuracy = np.mean(gt_mask == pred_mask)\n        \n        # Boundary F1-Score\n        boundary_f1 = self._boundary_f1_score(gt_binary, pred_binary)\n        \n        # Number of segments\n        n_segments = len(np.unique(pred_mask)) - 1  # Exclude background\n        \n        # Overall segmentation quality (weighted combination)\n        seg_quality = 0.4 * iou + 0.3 * dice + 0.2 * accuracy + 0.1 * boundary_f1\n        \n        return {\n            'iou': iou,\n            'dice': dice,\n            'accuracy': accuracy,\n            'boundary_f1': boundary_f1,\n            'n_segments': n_segments,\n            'seg_quality': seg_quality\n        }\n    \n    def _boundary_f1_score(self, gt_binary, pred_binary):\n        \"\"\"Calculate boundary F1-score\"\"\"\n        # Extract boundaries\n        gt_boundary = cv2.Canny(gt_binary.astype(np.uint8) * 255, 50, 150) > 0\n        pred_boundary = cv2.Canny(pred_binary.astype(np.uint8) * 255, 50, 150) > 0\n        \n        # Calculate precision and recall\n        if pred_boundary.sum() == 0:\n            precision = 0\n        else:\n            precision = np.logical_and(gt_boundary, pred_boundary).sum() / pred_boundary.sum()\n        \n        if gt_boundary.sum() == 0:\n            recall = 0\n        else:\n            recall = np.logical_and(gt_boundary, pred_boundary).sum() / gt_boundary.sum()\n        \n        # F1-score\n        if precision + recall == 0:\n            f1 = 0\n        else:\n            f1 = 2 * precision * recall / (precision + recall)\n        \n        return f1\n    \n    def _visualize_comprehensive_comparison(self, image, gt_mask, results):\n        \"\"\"Visualize comprehensive comparison of all methods\"\"\"\n        n_methods = len(results)\n        n_cols = min(4, n_methods + 2)\n        n_rows = (n_methods + 3) // n_cols\n        \n        fig, axes = plt.subplots(n_rows, n_cols, figsize=(5*n_cols, 5*n_rows))\n        if n_rows == 1:\n            axes = axes.reshape(1, -1)\n        \n        # Original and GT\n        axes[0, 0].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))\n        axes[0, 0].set_title('Original Image')\n        axes[0, 0].axis('off')\n        \n        axes[0, 1].imshow(gt_mask, cmap='tab10')\n        axes[0, 1].set_title('Ground Truth')\n        axes[0, 1].axis('off')\n        \n        # Results\n        idx = 2\n        for method_name, result in results.items():\n            row = idx // n_cols\n            col = idx % n_cols\n            \n            if row < n_rows and col < n_cols:\n                axes[row, col].imshow(result, cmap='tab10')\n                axes[row, col].set_title(method_name)\n                axes[row, col].axis('off')\n            \n            idx += 1\n        \n        # Hide unused subplots\n        for i in range(idx, n_rows * n_cols):\n            row = i // n_cols\n            col = i % n_cols\n            if row < n_rows and col < n_cols:\n                axes[row, col].axis('off')\n        \n        plt.tight_layout()\n        plt.show()\n\n# Initialize advanced techniques\nprint(\"🚀 Initializing Advanced Segmentation Techniques\")\nprint(\"=\" * 49)\n\nadvanced_seg = AdvancedSegmentationTechniques(dataset_generator.images, dataset_generator.masks)\n\n# Test advanced methods\nprint(\"\\n1️⃣  Testing SAM Simulation\")\nadvanced_seg.segment_anything_simulation(image_idx=0, num_points=15)\n\nprint(\"\\n2️⃣  Testing Panoptic Segmentation\")\nadvanced_seg.panoptic_segmentation(image_idx=1)\n\nprint(\"\\n3️⃣  Testing Real-Time Methods\")\nadvanced_seg.real_time_segmentation(image_idx=2)\n\nprint(\"\\n4️⃣  Testing Video Object Segmentation\")\nadvanced_seg.video_object_segmentation_simulation(num_frames=4)\n\nprint(\"\\n5️⃣  Comprehensive Evaluation\")\nadvanced_seg.comprehensive_evaluation(image_idx=0)

# 🎯 Final Comprehensive Demonstration

Let's bring everything together with a complete segmentation pipeline comparison.

In [ ]:
class ComprehensiveSegmentationDemo:
    """
    Complete demonstration of all segmentation methods
    """
    
    def __init__(self, images, masks):
        self.images = images
        self.masks = masks
        self.classical_seg = ClassicalSegmentation(images, masks)
        self.dl_seg = DeepLearningSegmentation(images, masks)
        self.advanced_seg = AdvancedSegmentationTechniques(images, masks)
        
        self.all_results = {}
        self.performance_summary = {}
    
    def run_complete_pipeline(self, image_idx=0, quick_demo=True):
        """Run complete segmentation pipeline on a single image"""
        print("🎯 COMPLETE SEGMENTATION PIPELINE DEMONSTRATION")
        print("=" * 52)
        print(f"📸 Processing Image {image_idx + 1}: {['Street', 'Indoor', 'Park', 'Office', 'Living Room'][image_idx]} Scene")
        print("=" * 52)
        
        image = self.images[image_idx]
        gt_mask = self.masks[image_idx]
        
        # === CLASSICAL METHODS ===
        print("\\n🔧 CLASSICAL METHODS")
        print("-" * 21)
        
        # Threshold segmentation
        thresh_result = self.classical_seg.threshold_segmentation(image_idx, show_steps=False)
        self.all_results['Threshold'] = thresh_result['multi_level']\n        \n        # Watershed\n        watershed_result = self.classical_seg.watershed_segmentation(image_idx, show_steps=False)\n        self.all_results['Watershed'] = watershed_result['segmented']\n        \n        # Region Growing\n        region_result = self.classical_seg.region_growing(image_idx, show_steps=False)\n        self.all_results['Region Growing'] = region_result['segmented']\n        \n        # GrabCut\n        grabcut_result = self.classical_seg.grabcut_segmentation(image_idx, show_steps=False)\n        self.all_results['GrabCut'] = grabcut_result['segmented']\n        \n        print(\"✅ Classical methods completed\")\n        \n        # === DEEP LEARNING METHODS ===\n        print(\"\\n🧠 DEEP LEARNING METHODS\")\n        print(\"-\" * 25)\n        \n        if quick_demo:\n            # Quick demo with pretrained models\n            try:\n                # DeepLabV3+\n                deeplabv3_result = self.dl_seg.segment_with_deeplabv3(image_idx)\n                self.all_results['DeepLabV3+'] = deeplabv3_result\n                print(\"✅ DeepLabV3+ segmentation completed\")\n            except Exception as e:\n                print(f\"⚠️  DeepLabV3+ failed: {e}\")\n                self.all_results['DeepLabV3+'] = np.zeros_like(gt_mask)\n            \n            try:\n                # Mask R-CNN\n                mask_rcnn_result, _ = self.dl_seg.segment_with_mask_rcnn(image_idx)\n                self.all_results['Mask R-CNN'] = mask_rcnn_result\n                print(\"✅ Mask R-CNN segmentation completed\")\n            except Exception as e:\n                print(f\"⚠️  Mask R-CNN failed: {e}\")\n                self.all_results['Mask R-CNN'] = np.zeros_like(gt_mask)\n        else:\n            # Full training (for production use)\n            print(\"🏋️  Training U-Net...\")\n            self.dl_seg.train_unet(epochs=10)\n            unet_result = self.dl_seg.segment_with_unet(image_idx, 'unet')\n            self.all_results['U-Net'] = unet_result\n            \n            print(\"🏋️  Training Attention U-Net...\")\n            self.dl_seg.train_attention_unet(epochs=10)\n            att_unet_result = self.dl_seg.segment_with_unet(image_idx, 'attention_unet')\n            self.all_results['Attention U-Net'] = att_unet_result\n        \n        # === ADVANCED METHODS ===\n        print(\"\\n🚀 ADVANCED METHODS\")\n        print(\"-\" * 18)\n        \n        # SAM Simulation\n        sam_result = self.advanced_seg.segment_anything_simulation(image_idx, num_points=12)\n        self.all_results['SAM Simulation'] = sam_result\n        \n        # Panoptic Segmentation\n        panoptic_result = self.advanced_seg.panoptic_segmentation(image_idx)\n        self.all_results['Panoptic'] = panoptic_result\n        \n        # Real-time methods\n        rt_super, rt_cluster, rt_edge = self.advanced_seg.real_time_segmentation(image_idx)\n        self.all_results['RT Superpixels'] = rt_super\n        self.all_results['RT Clustering'] = rt_cluster\n        \n        print(\"✅ Advanced methods completed\")\n        \n        # === COMPREHENSIVE EVALUATION ===\n        print(\"\\n📊 COMPREHENSIVE EVALUATION\")\n        print(\"-\" * 28)\n        \n        self._evaluate_all_methods(image, gt_mask)\n        \n        # === VISUALIZATION ===\n        print(\"\\n🎨 VISUALIZATION\")\n        print(\"-\" * 15)\n        \n        self._create_comprehensive_visualization(image, gt_mask)\n        \n        # === SUMMARY ===\n        self._create_performance_summary()\n        \n        return self.all_results\n    \n    def _evaluate_all_methods(self, image, gt_mask):\n        \"\"\"Evaluate all segmentation methods\"\"\"\n        \n        for method_name, pred_mask in self.all_results.items():\n            # Ensure same size\n            if pred_mask.shape != gt_mask.shape:\n                pred_mask = cv2.resize(pred_mask.astype(np.uint8), \n                                     (gt_mask.shape[1], gt_mask.shape[0]), \n                                     interpolation=cv2.INTER_NEAREST)\n            \n            # Calculate metrics\n            metrics = self._compute_comprehensive_metrics(gt_mask, pred_mask)\n            self.performance_summary[method_name] = metrics\n            \n            print(f\"{method_name:15}: IoU={metrics['iou']:.3f} | Dice={metrics['dice']:.3f} | Acc={metrics['accuracy']:.3f} | Segs={metrics['n_segments']:2d}\")\n    \n    def _compute_comprehensive_metrics(self, gt_mask, pred_mask):\n        \"\"\"Compute comprehensive evaluation metrics\"\"\"\n        # Binary masks for IoU/Dice\n        gt_binary = (gt_mask > 0).astype(int)\n        pred_binary = (pred_mask > 0).astype(int)\n        \n        # IoU\n        intersection = np.logical_and(gt_binary, pred_binary).sum()\n        union = np.logical_or(gt_binary, pred_binary).sum()\n        iou = intersection / union if union > 0 else 0.0\n        \n        # Dice\n        dice = 2 * intersection / (gt_binary.sum() + pred_binary.sum()) if (gt_binary.sum() + pred_binary.sum()) > 0 else 0.0\n        \n        # Accuracy\n        accuracy = np.mean(gt_mask == pred_mask)\n        \n        # Number of segments\n        n_segments = len(np.unique(pred_mask)) - 1\n        \n        # Boundary accuracy\n        gt_edges = cv2.Canny(gt_mask.astype(np.uint8), 1, 2)\n        pred_edges = cv2.Canny(pred_mask.astype(np.uint8), 1, 2)\n        boundary_acc = np.mean(gt_edges == pred_edges)\n        \n        # Overall quality score\n        quality_score = 0.3 * iou + 0.3 * dice + 0.2 * accuracy + 0.2 * boundary_acc\n        \n        return {\n            'iou': iou,\n            'dice': dice,\n            'accuracy': accuracy,\n            'boundary_accuracy': boundary_acc,\n            'n_segments': n_segments,\n            'quality_score': quality_score\n        }\n    \n    def _create_comprehensive_visualization(self, image, gt_mask):\n        \"\"\"Create comprehensive visualization of all results\"\"\"\n        n_methods = len(self.all_results)\n        n_cols = 4\n        n_rows = (n_methods + 3) // n_cols  # +2 for original and GT\n        \n        fig, axes = plt.subplots(n_rows, n_cols, figsize=(20, 5*n_rows))\n        if n_rows == 1:\n            axes = axes.reshape(1, -1)\n        \n        # Original image\n        axes[0, 0].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))\n        axes[0, 0].set_title('Original Image', fontsize=14, fontweight='bold')\n        axes[0, 0].axis('off')\n        \n        # Ground truth\n        axes[0, 1].imshow(gt_mask, cmap='tab10')\n        axes[0, 1].set_title('Ground Truth', fontsize=14, fontweight='bold')\n        axes[0, 1].axis('off')\n        \n        # Method results\n        idx = 2\n        for method_name, result in self.all_results.items():\n            row = idx // n_cols\n            col = idx % n_cols\n            \n            if row < n_rows:\n                # Get quality score for title\n                quality = self.performance_summary.get(method_name, {}).get('quality_score', 0)\n                \n                axes[row, col].imshow(result, cmap='tab10')\n                axes[row, col].set_title(f'{method_name}\\n(Q={quality:.3f})', fontsize=12)\n                axes[row, col].axis('off')\n            \n            idx += 1\n        \n        # Hide unused subplots\n        for i in range(idx, n_rows * n_cols):\n            row = i // n_cols\n            col = i % n_cols\n            if row < n_rows:\n                axes[row, col].axis('off')\n        \n        plt.suptitle('Comprehensive Segmentation Results Comparison', fontsize=18, fontweight='bold')\n        plt.tight_layout()\n        plt.show()\n    \n    def _create_performance_summary(self):\n        \"\"\"Create performance summary and rankings\"\"\"\n        print(\"\\n🏆 PERFORMANCE RANKINGS\")\n        print(\"=\" * 23)\n        \n        # Sort by quality score\n        sorted_methods = sorted(self.performance_summary.items(), \n                              key=lambda x: x[1]['quality_score'], reverse=True)\n        \n        print(\"\\n🥇 Overall Quality Rankings:\")\n        for i, (method, metrics) in enumerate(sorted_methods, 1):\n            medal = [\"🥇\", \"🥈\", \"🥉\"][min(i-1, 2)] if i <= 3 else f\"{i:2d}.\"\n            print(f\"{medal} {method:15}: {metrics['quality_score']:.3f}\")\n        \n        # Best in each category\n        print(\"\\n🎯 Category Leaders:\")\n        \n        best_iou = max(self.performance_summary.items(), key=lambda x: x[1]['iou'])\n        print(f\"📐 Best IoU:           {best_iou[0]:15} ({best_iou[1]['iou']:.3f})\")\n        \n        best_dice = max(self.performance_summary.items(), key=lambda x: x[1]['dice'])\n        print(f\"🎲 Best Dice:          {best_dice[0]:15} ({best_dice[1]['dice']:.3f})\")\n        \n        best_acc = max(self.performance_summary.items(), key=lambda x: x[1]['accuracy'])\n        print(f\"🎯 Best Accuracy:      {best_acc[0]:15} ({best_acc[1]['accuracy']:.3f})\")\n        \n        best_boundary = max(self.performance_summary.items(), key=lambda x: x[1]['boundary_accuracy'])\n        print(f\"🔲 Best Boundary:      {best_boundary[0]:15} ({best_boundary[1]['boundary_accuracy']:.3f})\")\n        \n        # Method characteristics\n        print(\"\\n⚡ Method Characteristics:\")\n        classical_methods = ['Threshold', 'Watershed', 'Region Growing', 'GrabCut']\n        dl_methods = ['U-Net', 'Attention U-Net', 'DeepLabV3+', 'Mask R-CNN']\n        advanced_methods = ['SAM Simulation', 'Panoptic', 'RT Superpixels', 'RT Clustering']\n        \n        for category, methods in [('Classical', classical_methods), \n                                ('Deep Learning', dl_methods), \n                                ('Advanced', advanced_methods)]:\n            category_methods = [m for m in methods if m in self.performance_summary]\n            if category_methods:\n                avg_quality = np.mean([self.performance_summary[m]['quality_score'] for m in category_methods])\n                print(f\"   {category:12}: Avg Quality = {avg_quality:.3f} ({len(category_methods)} methods)\")\n        \n        # Create performance comparison chart\n        self._plot_performance_comparison()\n    \n    def _plot_performance_comparison(self):\n        \"\"\"Plot performance comparison chart\"\"\"\n        methods = list(self.performance_summary.keys())\n        iou_scores = [self.performance_summary[m]['iou'] for m in methods]\n        dice_scores = [self.performance_summary[m]['dice'] for m in methods]\n        accuracy_scores = [self.performance_summary[m]['accuracy'] for m in methods]\n        quality_scores = [self.performance_summary[m]['quality_score'] for m in methods]\n        \n        # Create subplots\n        fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))\n        \n        # IoU comparison\n        bars1 = ax1.bar(range(len(methods)), iou_scores, color='skyblue', alpha=0.8)\n        ax1.set_title('IoU (Intersection over Union)', fontsize=14, fontweight='bold')\n        ax1.set_ylabel('IoU Score')\n        ax1.set_xticks(range(len(methods)))\n        ax1.set_xticklabels(methods, rotation=45, ha='right')\n        ax1.grid(True, alpha=0.3)\n        \n        # Add value labels on bars\n        for bar, score in zip(bars1, iou_scores):\n            ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, \n                    f'{score:.3f}', ha='center', va='bottom', fontsize=8)\n        \n        # Dice comparison\n        bars2 = ax2.bar(range(len(methods)), dice_scores, color='lightgreen', alpha=0.8)\n        ax2.set_title('Dice Coefficient', fontsize=14, fontweight='bold')\n        ax2.set_ylabel('Dice Score')\n        ax2.set_xticks(range(len(methods)))\n        ax2.set_xticklabels(methods, rotation=45, ha='right')\n        ax2.grid(True, alpha=0.3)\n        \n        for bar, score in zip(bars2, dice_scores):\n            ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, \n                    f'{score:.3f}', ha='center', va='bottom', fontsize=8)\n        \n        # Accuracy comparison\n        bars3 = ax3.bar(range(len(methods)), accuracy_scores, color='lightcoral', alpha=0.8)\n        ax3.set_title('Pixel Accuracy', fontsize=14, fontweight='bold')\n        ax3.set_ylabel('Accuracy Score')\n        ax3.set_xticks(range(len(methods)))\n        ax3.set_xticklabels(methods, rotation=45, ha='right')\n        ax3.grid(True, alpha=0.3)\n        \n        for bar, score in zip(bars3, accuracy_scores):\n            ax3.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, \n                    f'{score:.3f}', ha='center', va='bottom', fontsize=8)\n        \n        # Overall quality comparison\n        bars4 = ax4.bar(range(len(methods)), quality_scores, color='gold', alpha=0.8)\n        ax4.set_title('Overall Quality Score', fontsize=14, fontweight='bold')\n        ax4.set_ylabel('Quality Score')\n        ax4.set_xticks(range(len(methods)))\n        ax4.set_xticklabels(methods, rotation=45, ha='right')\n        ax4.grid(True, alpha=0.3)\n        \n        for bar, score in zip(bars4, quality_scores):\n            ax4.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, \n                    f'{score:.3f}', ha='center', va='bottom', fontsize=8)\n        \n        plt.suptitle('Comprehensive Performance Comparison', fontsize=16, fontweight='bold')\n        plt.tight_layout()\n        plt.show()\n    \n    def test_all_scenes(self, quick_demo=True):\n        \"\"\"Test segmentation pipeline on all scenes\"\"\"\n        print(\"🌟 TESTING ALL SCENES\")\n        print(\"=\" * 21)\n        \n        scene_names = ['Street Scene', 'Indoor Scene', 'Park Scene', 'Office Scene', 'Living Room Scene']\n        all_scene_results = {}\n        \n        for i, scene_name in enumerate(scene_names):\n            print(f\"\\n🎬 Processing {scene_name} (Image {i+1}/5)\")\n            print(\"-\" * (15 + len(scene_name)))\n            \n            # Run pipeline on this scene\n            scene_results = self.run_complete_pipeline(image_idx=i, quick_demo=quick_demo)\n            all_scene_results[scene_name] = {\n                'results': scene_results,\n                'performance': self.performance_summary.copy()\n            }\n            \n            # Clear results for next scene\n            self.all_results = {}\n            self.performance_summary = {}\n        \n        # Aggregate analysis\n        self._analyze_across_scenes(all_scene_results)\n        \n        return all_scene_results\n    \n    def _analyze_across_scenes(self, all_scene_results):\n        \"\"\"Analyze performance across different scenes\"\"\"\n        print(\"\\n🔍 CROSS-SCENE ANALYSIS\")\n        print(\"=\" * 23)\n        \n        # Collect all method performances\n        method_performances = {}\n        \n        for scene_name, scene_data in all_scene_results.items():\n            for method_name, metrics in scene_data['performance'].items():\n                if method_name not in method_performances:\n                    method_performances[method_name] = []\n                method_performances[method_name].append(metrics['quality_score'])\n        \n        # Calculate average performance\n        print(\"📊 Average Performance Across All Scenes:\")\n        avg_performances = []\n        for method_name, scores in method_performances.items():\n            avg_score = np.mean(scores)\n            std_score = np.std(scores)\n            avg_performances.append((method_name, avg_score, std_score))\n        \n        # Sort by average performance\n        avg_performances.sort(key=lambda x: x[1], reverse=True)\n        \n        for i, (method, avg, std) in enumerate(avg_performances, 1):\n            medal = [\"🥇\", \"🥈\", \"🥉\"][min(i-1, 2)] if i <= 3 else f\"{i:2d}.\"\n            print(f\"{medal} {method:15}: {avg:.3f} ± {std:.3f}\")\n        \n        # Scene difficulty analysis\n        print(\"\\n🎯 Scene Difficulty Analysis:\")\n        scene_difficulties = []\n        for scene_name, scene_data in all_scene_results.items():\n            scene_avg = np.mean([metrics['quality_score'] for metrics in scene_data['performance'].values()])\n            scene_difficulties.append((scene_name, scene_avg))\n        \n        scene_difficulties.sort(key=lambda x: x[1])\n        \n        for i, (scene, difficulty) in enumerate(scene_difficulties):\n            difficulty_level = [\"😰 Very Hard\", \"😟 Hard\", \"😐 Medium\", \"😊 Easy\", \"😄 Very Easy\"][min(int(difficulty * 5), 4)]\n            print(f\"{scene:15}: {difficulty:.3f} {difficulty_level}\")\n\n# 🎬 FINAL DEMONSTRATION\nprint(\"🌟\" * 25)\nprint(\"🎬 FINAL COMPREHENSIVE SEGMENTATION DEMONSTRATION\")\nprint(\"🌟\" * 25)\n\n# Initialize comprehensive demo\ndemo = ComprehensiveSegmentationDemo(\n    dataset_generator.images, \n    dataset_generator.masks\n)\n\n# Run complete demonstration\nprint(\"\\n🚀 Running Complete Pipeline on Street Scene...\")\nresults = demo.run_complete_pipeline(image_idx=0, quick_demo=True)\n\nprint(\"\\n\" + \"=\"*60)\nprint(\"🎯 SEGMENTATION NOTEBOOK COMPLETED SUCCESSFULLY!\")\nprint(\"=\"*60)\nprint(\"\\n📋 What we accomplished:\")\nprint(\"   ✅ Created realistic multi-object datasets (8+ object classes)\")\nprint(\"   ✅ Implemented classical segmentation methods\")\nprint(\"   ✅ Built deep learning segmentation models\")\nprint(\"   ✅ Explored advanced techniques (SAM, Panoptic, Real-time)\")\nprint(\"   ✅ Comprehensive evaluation with multiple metrics\")\nprint(\"   ✅ Performance comparison across all methods\")\nprint(\"\\n🎉 Ready for production use and further development!\")